In [10]:
import pandas as pd
import numpy as np
import plotly as pt
import seaborn as sns
!pip install pymatgen
!pip install mp_api
import requests
import json

In [11]:
update_IDs = False
target_col = 'Log_rate'
cross_validation = True
dataset_from_google_drive = True

df = None
if(dataset_from_google_drive):
  df = pd.read_excel("/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/Perovskite dataset export.xlsx",sheet_name='Photocatalytic dataset')
else:
  df = pd.read_excel("dataset.xlsx")

In [12]:
df

,Perovskite,Hill formula,Interlayer space composition,"Bandgap, eV",DOI,Materials Project ID,COD_ID,Springer_ID,MP_CIF_modifier,COD_CIF_modifier,...,"CatW, g/L","Alcohol, %",Sac Agent 1,Sac agent 2,Light type,"Power, W","Wave length min, nm",Cell material,"Temperature, K","Rate, umol/(g*h)"
0,K4Nb6O17,K4 Nb6 O17,NaN,3.50,10.1007/BF00765319,mp-560692,1001842,NaN,NaN,NaN,...,2.85,14.3,-,-,Hg lamp,450.0,NaN,NaN,NaN,130.000000
1,KLaNb2O7,K La Nb2 O7,NaN,3.20,10.1007/BF00765319,mp-1223501,1545643,NaN,NaN,NaN,...,2.85,14.3,-,-,Hg lamp,450.0,NaN,NaN,NaN,28.000000
2,RbLaNb2O7,La Nb2 O7 Rb,NaN,3.35,10.1007/BF00765319,mp-553965,-1,NaN,NaN,NaN,...,2.85,14.3,-,-,Hg lamp,450.0,NaN,NaN,NaN,60.000000
3,CsLaNb2O7,Cs La Nb2 O7,NaN,3.30,10.1007/BF00765319,mp-553248,2004917,NaN,NaN,NaN,...,2.85,14.3,-,-,Hg lamp,450.0,NaN,NaN,NaN,12.000000
4,KCa2Nb3O10,Ca2 K Nb3 O10,NaN,3.35,10.1007/BF00765319,mp-557195,1521061,NaN,NaN,NaN,...,2.85,14.3,-,-,Hg lamp,450.0,NaN,NaN,NaN,14.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1084,TiO2,NaN,NaN,3.20,10.1134/S107036321404001X,mp-1245098,1010942,NaN,NaN,NaN,...,1.00,0.1,-,-,Hg lamp,125.0,220.0,quartz,288.0,38.181818
1085,TiO2,NaN,NaN,3.20,10.1134/S107036321404001X,mp-1245098,1010942,NaN,NaN,NaN,...,1.00,0.1,-,-,Hg lamp,125.0,220.0,quartz,288.0,36.363636
1086,TiO2,NaN,NaN,3.20,10.1134/S107036321404001X,mp-1245098,1010942,NaN,NaN,NaN,...,1.00,0.1,-,-,Hg lamp,125.0,220.0,quartz,288.0,9.090909
1087,TiO2,NaN,NaN,3.20,10.1134/S107036321404001X,mp-1245098,1010942,NaN,NaN,NaN,...,1.00,0.1,-,-,Hg lamp,125.0,220.0,quartz,288.0,9.090909


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1089 entries, 0 to 1088
Data columns (total 88 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Perovskite                            1089 non-null   object 
 1   Hill formula                          730 non-null    object 
 2   Interlayer space composition          5 non-null      object 
 3   Bandgap, eV                           1054 non-null   float64
 4   DOI                                   1084 non-null   object 
 5   Materials Project ID                  1058 non-null   object 
 6   COD_ID                                935 non-null    object 
 7   Springer_ID                           290 non-null    object 
 8   MP_CIF_modifier                       499 non-null    object 
 9   COD_CIF_modifier                      197 non-null    object 
 10  Springer_CIF_modifier                 121 non-null    object 
 11  Materials Project

In [14]:
df.describe()

,"Bandgap, eV",Z,Z_MP,Z_COD,Z_Springer,"d,A",a_MP,b_MP,c_MP,a_COD,...,Oxygen_concentration avg,Packing fraction avg,CalcT(K),Calc time (h),"CatW, g/L","Alcohol, %","Power, W","Wave length min, nm","Temperature, K","Rate, umol/(g*h)"
count,1054.000000,694.000000,832.000000,836.000000,836.000000,807.000000,827.000000,354.000000,354.000000,832.000000,...,832.000000,832.000000,1010.000000,973.000000,987.000000,966.000000,994.000000,689.000000,540.000000,989.000000
mean,3.313241,2.275216,1.260817,1.281100,0.151715,13.707422,3.084536,7.406617,14.790353,3.040421,...,inf,0.238767,1287.020792,19.338386,1.428618,5.628230,306.726358,288.388970,294.435185,6914.999659
std,0.644047,1.762830,3.460696,2.116075,0.642689,8.610255,4.236046,3.399894,8.264087,6.154639,...,NaN,0.257556,222.542464,25.264723,1.324191,8.781436,198.606250,88.814702,13.175195,21648.175384
min,0.010000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,3.840000,3.904344,0.000000,...,0.000000,0.000000,453.000000,0.000000,0.000000,0.000000,100.000000,200.000000,288.000000,0.000000
25%,3.100000,1.000000,0.000000,0.000000,0.000000,10.548300,0.000000,3.950634,10.502783,0.000000,...,0.037279,0.000000,1223.000000,10.000000,0.500000,0.000000,125.000000,220.000000,288.000000,15.500000
50%,3.410000,2.000000,0.000000,0.000000,0.000000,14.733000,0.000000,7.415443,12.879040,0.000000,...,0.042938,0.000000,1373.000000,12.000000,1.000000,1.000000,300.000000,220.000000,288.000000,166.000000
75%,3.600000,4.000000,1.000000,2.000000,0.000000,17.900000,5.801718,7.938952,15.513552,3.899000,...,0.046684,0.533675,1423.000000,24.000000,2.000000,10.000000,450.000000,400.000000,293.000000,1680.000000
max,5.500000,16.000000,50.000000,16.000000,4.000000,38.670000,22.855297,20.551678,48.809404,41.857000,...,inf,0.657822,1873.000000,200.000000,10.000000,90.000000,1000.000000,660.000000,333.000000,156000.000000


In [15]:
df.columns

Index(['Perovskite', 'Hill formula', 'Interlayer space composition',
       'Bandgap, eV', 'DOI', 'Materials Project ID', 'COD_ID', 'Springer_ID',
       'MP_CIF_modifier', 'COD_CIF_modifier', 'Springer_CIF_modifier',
       'Materials Project verification', 'COD verification',
       'Springer verification', 'General verification', 'MP_CIF_modified',
       'COD_CIF_modified', 'Springer_CIF_modified', 'Z', 'Z_MP', 'Z_COD',
       'Z_Springer', 'a, A', 'b, A', 'c, A', 'Symmetry group', 'd,A', 'a_MP',
       'b_MP', 'c_MP', 'a_COD', 'b_COD', 'c_COD', 'a_Springer', 'b_Springer',
       'c_Springer', 'Number of octahedrons on a layer', 'Valence electrons',
       'Volume', 'Volume_MP', 'Volume_COD', 'Volume_Springer',
       'Valence Electrons Density', 'Valence Electrons Density_MP',
       'Valence Electrons Density_COD', 'Springer_Valence Electrons Density',
       'avg s valence electrons', 'avg p valence electrons',
       'avg d valence electrons', 'avg f valence electrons',
       

In [16]:
init_columns = ['Perovskite', 'Interlayer space composition',
       'Bandgap, eV', 'Materials Project ID', 'COD_ID', 'Springer_ID',
       'Z', 'a, A', 'b, A', 'c, A',
       'Symmetry group', 'd,A',
       'Number of octahedrons on a layer', 'Prep Method', 'CalcT(K)', 'Calc time (h)',
       'Nitrogen', 'Promotion method', 'Promoter', 'Promoter, w%',
       'Surface area, m2/g', 'CatW, g/L', 'Alcohol, %', 'Sac Agent 1',
       'Sac agent 2', 'Light type', 'Power, W', 'Wave length min, nm',
       'Cell material', 'Temperature, K', 'Rate, umol/(g*h)']

In [17]:
df = df[init_columns]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1089 entries, 0 to 1088
Data columns (total 31 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Perovskite                        1089 non-null   object 
 1   Interlayer space composition      5 non-null      object 
 2   Bandgap, eV                       1054 non-null   float64
 3   Materials Project ID              1058 non-null   object 
 4   COD_ID                            935 non-null    object 
 5   Springer_ID                       290 non-null    object 
 6   Z                                 694 non-null    float64
 7   a, A                              735 non-null    object 
 8   b, A                              717 non-null    object 
 9   c, A                              717 non-null    object 
 10  Symmetry group                    494 non-null    object 
 11  d,A                               807 non-null    float64
 12  Number

#Formatting


In [18]:
subs_map = {
    "Ph": "C6H5",
    "Bn": "C7H7",
    #"Pr": "C3H7",
    "Bu": "C4H9",
    "Hx": "C6H13",
    "Me": "CH3",
    "Et": "C2H5",
    "Oc": "C8H17",
    "Dc": "C10H21",
}

import re

def expand_substituents(formula):
    if pd.isna(formula):
        return formula

    for abbr, full in subs_map.items():
        formula = re.sub(rf'{abbr}', full, formula)
    return formula

In [19]:
print(df.shape[0])
df = df[~df['Perovskite'].str.contains("Nx", na=False)]
df = df[~df['Perovskite'].str.contains("Ox", na=False)]
print(df.shape[0])

1089
1076


In [20]:
df['Perovskite'] = df['Perovskite'].apply(expand_substituents)
#print(expand_substituents('HNdTiO4*MeNH2'))

In [21]:
df = df.replace("-", np.nan)
df["a, A"] = df["a, A"].astype("float64")
df["b, A"] = df["b, A"].astype("float64")
df["c, A"] = df["c, A"].astype("float64")
df["Number of octahedrons on a layer"] = df["Number of octahedrons on a layer"].astype("float64")

/tmp/ipython-input-3678714408.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace("-", np.nan)


In [22]:
df["Nitrogen"] = df["Nitrogen"].fillna(False)
df["Nitrogen"]
df["Nitrogen"].astype("bool")
df["Nitrogen"].value_counts()

/tmp/ipython-input-1068474039.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["Nitrogen"] = df["Nitrogen"].fillna(False)


,count
Nitrogen,
False,1029
True,47


In [23]:
df["Promoter, w%"] = df["Promoter, w%"].fillna(0)
df["Promoter, w%"].value_counts()

,count
"Promoter, w%",
0.000,493
1.000,295
0.100,81
0.500,72
0.150,27
0.200,24
0.300,16
3.000,15
2.000,10


In [24]:
df["Surface area, m2/g"] = df["Surface area, m2/g"].astype("float64")
df["Surface area, m2/g"].value_counts()

#df['Materials Project ID'] = df['Materials Project ID'].astype(str)
print(df['Materials Project ID'].dtype)

object


In [25]:
df["Perovskite"].value_counts()

,count
Perovskite,
HCa2Nb3O10,33
La2Ti2O7,32
H2Nd2Ti3O10*2C4H9NH2,32
TiO2,30
K4Nb6O17,28
...,...
Na2La2Mn0.9Ti2.1O10,1
Na2La2MnTi2O10,1
H0.4K1.6La2Ti3O10*1.07H2O,1


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1076 entries, 0 to 1088
Data columns (total 31 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Perovskite                        1076 non-null   object 
 1   Interlayer space composition      5 non-null      object 
 2   Bandgap, eV                       1041 non-null   float64
 3   Materials Project ID              1045 non-null   object 
 4   COD_ID                            922 non-null    object 
 5   Springer_ID                       290 non-null    object 
 6   Z                                 694 non-null    float64
 7   a, A                              731 non-null    float64
 8   b, A                              713 non-null    float64
 9   c, A                              713 non-null    float64
 10  Symmetry group                    494 non-null    object 
 11  d,A                               796 non-null    float64
 12  Number of o

In [27]:
df.to_excel("checkpoint_formatting.xlsx")


#Data cleaning

##Missing data handling

In [ ]:
df.isna().sum()

,0
Perovskite,0
Interlayer space composition,1063
"Bandgap, eV",36
Materials Project ID,188
COD_ID,213
Springer_ID,887
Z,382
"a, A",345
"b, A",363
"c, A",363


In [28]:
df.columns

Index(['Perovskite', 'Interlayer space composition', 'Bandgap, eV',
       'Materials Project ID', 'COD_ID', 'Springer_ID', 'Z', 'a, A', 'b, A',
       'c, A', 'Symmetry group', 'd,A', 'Number of octahedrons on a layer',
       'Prep Method', 'CalcT(K)', 'Calc time (h)', 'Nitrogen',
       'Promotion method', 'Promoter', 'Promoter, w%', 'Surface area, m2/g',
       'CatW, g/L', 'Alcohol, %', 'Sac Agent 1', 'Sac agent 2', 'Light type',
       'Power, W', 'Wave length min, nm', 'Cell material', 'Temperature, K',
       'Rate, umol/(g*h)'],
      dtype='object')

In [29]:
drop_columns = ['Perovskite', 'Bandgap, eV',
       'Prep Method', 'CalcT(K)', 'Calc time (h)',
       'CatW, g/L', 'Light type',
       'Power, W',
       'Rate, umol/(g*h)']

drop_columns=['Perovskite','Rate, umol/(g*h)']
df['Bandgap, eV'] = df['Bandgap, eV'].fillna(df['Bandgap, eV'].mean())
df['Prep Method'] = df['Prep Method'].fillna(df['Prep Method'].mode()[0])
df['CalcT(K)'] = df['CalcT(K)'].fillna(df['CalcT(K)'].mean())
df['Calc time (h)'] = df['Calc time (h)'].fillna(df['Calc time (h)'].mean())
df['CatW, g/L'] = df['CatW, g/L'].fillna(df['CatW, g/L'].mean())
df['Light type'] = df['Light type'].fillna(df['Light type'].mode()[0])
df['Power, W'] = df['Power, W'].fillna(df['Power, W'].mean())

df.dropna(subset=drop_columns, inplace=True)
df.shape[0]

976

In [30]:
df.isna().sum()

,0
Perovskite,0
Interlayer space composition,971
"Bandgap, eV",0
Materials Project ID,31
COD_ID,150
Springer_ID,715
Z,361
"a, A",329
"b, A",344
"c, A",344


In [31]:
def calculate_average_specific_area(dataframe):
  total=0;
  n=0;
  for index, row in df.iterrows():
    area =  row['Surface area, m2/g']
    if(np.isnan(area) or area > 55):
      continue
    total += area
    n+=1
  return total/n

In [32]:
avr_specific_area = calculate_average_specific_area(df)
print(avr_specific_area)
df["Surface area, m2/g"] = df["Surface area, m2/g"].fillna(avr_specific_area)

df["Alcohol, %"] = df["Alcohol, %"].fillna(0)
df["Wave length min, nm"] = df["Wave length min, nm"].fillna(df["Wave length min, nm"].min())
df["Cell material"] = df["Cell material"].fillna(df["Cell material"].mode()[0])
print(df["Cell material"].mode()[0])
df["Temperature, K"] = df["Temperature, K"].fillna(298.15)
df["Materials Project ID"] = df["Materials Project ID"].fillna(-1)
df["COD_ID"] = df["COD_ID"].fillna(-1)
df["Springer_ID"] = df["Springer_ID"].fillna(-1)

10.128147826086945
quartz


In [33]:
df["Promoter"] = df["Promoter"].fillna("No promoter")
df["Promotion method"] = df["Promotion method"].fillna("No promotion")

In [34]:
df.isna().sum()

,0
Perovskite,0
Interlayer space composition,971
"Bandgap, eV",0
Materials Project ID,0
COD_ID,0
Springer_ID,0
Z,361
"a, A",329
"b, A",344
"c, A",344


## Duplicates handling

In [35]:
duplicates = df[ df.duplicated()]
duplicates

,Perovskite,Interlayer space composition,"Bandgap, eV",Materials Project ID,COD_ID,Springer_ID,Z,"a, A","b, A","c, A",...,"CatW, g/L","Alcohol, %",Sac Agent 1,Sac agent 2,Light type,"Power, W","Wave length min, nm",Cell material,"Temperature, K","Rate, umol/(g*h)"


##Handling outliers

In [36]:
columns_outliers_detedection = [ 'Bandgap, eV',
      'Calc time (h)', 'Promoter, w%', 'Surface area, m2/g',
       'CatW, g/L', 'Alcohol, %',
       'Power, W', 'Wave length min, nm', 'Temperature, K']
columns_outliers_detedection = [ 'Bandgap, eV',
      'Calc time (h)', 'Promoter, w%',
       'CatW, g/L',
       'Power, W', 'Wave length min, nm', 'Temperature, K']
#columns_outliers_detedection=[]

In [37]:
import plotly.express as px
def plot_destribution(df,column):
  #fig = px.histogram(df, x=column,nbins=40,width=800, height=600, title=column)
  fig = px.violin(df, x=column,width=600, height=600, title=column)
  fig.update_layout(font_size=20)
  fig.show()
def plot_histogram(df,column):
  fig = px.histogram(df, x=column,nbins=20,width=600, height=600, title=column)
  fig.update_layout(font_size=20)
  fig.show()

In [38]:
for column in columns_outliers_detedection:
  plot_histogram(df,column)

In [39]:
def IQR_column_outliers_removal(_df, _column):
  num_rows_before = _df.shape[0]
  Q1 = _df[_column].quantile(0.25)
  Q3 = _df[_column].quantile(0.75)
  IQR = Q3 - Q1

  lower_bound = Q1 - 1.5 * IQR
  upper_bound = Q3 + 1.5 * IQR

  df_clean = _df[(_df[_column]>= lower_bound) & (_df[_column] <= upper_bound)]
  num_rows_after = df_clean.shape[0]
  removed = num_rows_before - num_rows_after
  print(f'Column: {_column} removed ouliers: {removed }')
  print(f'min: {lower_bound} max: {upper_bound}')
  return df_clean

In [40]:
for column in columns_outliers_detedection:
  df = IQR_column_outliers_removal(df,column)
print(f'Rows left: {df.shape[0]}')

Column: Bandgap, eV removed ouliers: 159
min: 2.5525 max: 4.2285
Column: Calc time (h) removed ouliers: 107
min: -11.0 max: 45.0
Column: Promoter, w% removed ouliers: 25
min: -1.5 max: 2.5
Column: CatW, g/L removed ouliers: 26
min: -1.255 max: 3.425
Column: Power, W removed ouliers: 12
min: -287.5 max: 812.5
Column: Wave length min, nm removed ouliers: 131
min: 170.0 max: 250.0
Column: Temperature, K removed ouliers: 4
min: 272.77500000000003 max: 313.37499999999994
Rows left: 512


#Descriptors calculation



## Material Project IDs


In [41]:
from mp_api.client import MPRester

API_KEY = "GFsoU5OV3dEngGT860TOtWcn35bE4y6l"
with MPRester(API_KEY) as mpr:
    materials = mpr.materials.search(formula="KCa2Nb3O10")
for material in materials:
    print(material.material_id, material.formula_pretty)

Retrieving MaterialsDoc documents:   0%|          | 0/2 [00:00<?, ?it/s]

mp-557195 KCa2Nb3O10
mp-1223594 KCa2Nb3O10


In [42]:
def get_material_id_by_formula(f):
  with MPRester(API_KEY) as mpr:
    print(f"Composition: {f}")
    try:
      materials = mpr.materials.search(formula=f)
      if len(materials)>0:
        print(f"Composition: {f} ID is {materials[0].material_id}")
        return str(materials[0].material_id)
      else:
        print(f"Composition: {f} ID is not found")
        return str(-1)
    except Exception as e:
      print(f"Composition: {f} ID is not found")
      return str(-1)

In [43]:
def update_Materials_Project_IDs(_df):
  for index, row in _df.iterrows():
        if pd.isna(row['Materials Project ID']) or row['Materials Project ID']==-1 or row['Materials Project ID']==-2 :
            new_id = get_material_id_by_formula(row['Perovskite'])
            print(f"New ID fetched: {new_id}  for formula {row['Perovskite']}")
            _df.at[index, 'Materials Project ID'] = str(new_id)
  return _df

In [44]:
if update_IDs:
  df = update_Materials_Project_IDs(df)

##HIll's formula

In [45]:
import re
from collections import Counter
def parse_formula(formula):
    """Extracts element counts from a chemical formula."""
    pattern = r"([A-Z][a-z]*)(\d*)"
    matches = re.findall(pattern, formula)

    element_counts = Counter()
    for element, count in matches:
        element_counts[element] += int(count) if count else 1  # Default to 1 if count is missing

    return element_counts
def to_hill_notation(formula):
    """Converts a chemical formula to Hill notation."""
    element_counts = parse_formula(formula)

    # Hill notation rules
    if "C" in element_counts:
        elements_sorted = ["C", "H"] if "H" in element_counts else ["C"]
        remaining_elements = sorted(e for e in element_counts if e not in ["C", "H"])
        elements_sorted.extend(remaining_elements)
    else:
        elements_sorted = sorted(element_counts.keys())
    hill_formula = " ".join(f"{el}{element_counts[el] if element_counts[el] > 1 else ''}" for el in elements_sorted)
    #print(f':{hill_formula}:')
    return hill_formula


In [46]:
formulas = ["H2O", "C6H12O6", "NH3", "Fe2O3", "CH4", "CCl4", "H4C", "KCa2Nb3O10"]
for f in formulas:
    print(f"{f} -> {to_hill_notation(f)}")

H2O -> H2 O
C6H12O6 -> C6 H12 O6
NH3 -> H3 N
Fe2O3 -> Fe2 O3
CH4 -> C H4
CCl4 -> C Cl4
H4C -> C H4
KCa2Nb3O10 -> Ca2 K Nb3 O10


In [47]:
df['Hill formula']=df['Perovskite'].apply(to_hill_notation)
df

,Perovskite,Interlayer space composition,"Bandgap, eV",Materials Project ID,COD_ID,Springer_ID,Z,"a, A","b, A","c, A",...,"Alcohol, %",Sac Agent 1,Sac agent 2,Light type,"Power, W","Wave length min, nm",Cell material,"Temperature, K","Rate, umol/(g*h)",Hill formula
9,TiO2,NaN,3.0,mp-1245098,1010942,-1,NaN,NaN,NaN,NaN,...,14.3,NaN,NaN,Hg lamp,450.0,200.0,quartz,298.15,0.000000,O2 Ti
19,TiO2,NaN,3.0,mp-1245098,1010942,-1,NaN,NaN,NaN,NaN,...,14.3,NaN,NaN,Hg lamp,450.0,200.0,quartz,298.15,7400.000000,O2 Ti
69,RbLaTa2O7,NaN,3.9,mp-541600,-1,sd_1050391,1.0,3.885,3.885,11.12,...,0.0,NaN,NaN,Hg lamp,400.0,200.0,quartz,298.15,6.000000,La O7 Rb Ta2
70,RbPrTa2O7,NaN,3.5,-1,-1,sd_1050391,1.0,3.884,3.884,11.12,...,0.0,NaN,NaN,Hg lamp,400.0,200.0,quartz,298.15,4.500000,O7 Pr Rb Ta2
71,RbNdTa2O7,NaN,3.8,-1,-1,sd_1050391,1.0,3.846,3.846,11.10,...,0.0,NaN,NaN,Hg lamp,400.0,200.0,quartz,298.15,235.000000,Nd O7 Rb Ta2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1084,TiO2,NaN,3.2,mp-1245098,1010942,-1,NaN,NaN,NaN,NaN,...,0.1,NaN,NaN,Hg lamp,125.0,220.0,quartz,288.00,38.181818,O2 Ti
1085,TiO2,NaN,3.2,mp-1245098,1010942,-1,NaN,NaN,NaN,NaN,...,0.1,NaN,NaN,Hg lamp,125.0,220.0,quartz,288.00,36.363636,O2 Ti
1086,TiO2,NaN,3.2,mp-1245098,1010942,-1,NaN,NaN,NaN,NaN,...,0.1,NaN,NaN,Hg lamp,125.0,220.0,quartz,288.00,9.090909,O2 Ti
1087,TiO2,NaN,3.2,mp-1245098,1010942,-1,NaN,NaN,NaN,NaN,...,0.1,NaN,NaN,Hg lamp,125.0,220.0,quartz,288.00,9.090909,O2 Ti


##COD_IDs


In [48]:
def get_COD_ID_for_formula(formula):
  print(f"Formula :{formula}:")
  url=f"https://www.crystallography.net/cod/result?formula={formula}&format=json"
  response = requests.get(url)
  if response.status_code == 200:
    data = response.json()
    #print("Len: ",len(data))
    if(len(data)>0):
      #print(data[0]['file'])
      return data[0]['file']
  print(-1)
  return -1

def update_COD_ID(_df):
  for index, row in _df.iterrows():
        if pd.isna(row['COD_ID']) or row['COD_ID']==-1 or row['COD_ID']==-2 :
            print(f"New ID fetching start for formula {row['Hill formula']}")
            new_id = get_COD_ID_for_formula(row['Hill formula'])
            print(f"New ID fetched: {new_id}  for formula {row['Hill formula']}")
            _df.at[index, 'COD_ID'] = new_id
  return _df


In [49]:
df.to_excel('checkpoint_Hill_formula.xlsx')
#print(get_COD_ID("O2 Ti"))
if update_IDs:
  df = update_COD_ID(df)

##CIF files extraction

In [50]:
def extract_CIF_from_MP(material_id):
  if pd.isna(material_id) or material_id == "-1" or material_id == -1 or material_id == -2:
     print("Skipping invalid ID:", material_id)
     return
  print("ID: ", material_id)

  path = f'/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/CIF/{material_id}.cif'
  if os.path.exists(path):
    print(f'CIF file for {material_id} already exist')
    return

  with MPRester(API_KEY) as mpr:
    #data = mpr.materials.search(material_ids=material_id)
    structure = mpr.materials.get_structure_by_material_id(material_id )
    cif_string = structure.to("struct.cif")

  ##with open(f"{material_id}.cif", "w") as f:
  #    f.write(cif_string)
  with open(f'/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/CIF/{material_id}.cif', 'w') as f:
      f.write(cif_string)
  print("ID: ", material_id, " done!")

In [51]:
import os
def extract_cif_from_COD(COD_ID):
  if(COD_ID==-1 or COD_ID==-2 ):
    return
  print(COD_ID)
  path = f'/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/CIF/{COD_ID}.cif'
  if os.path.exists(path):
    print(f'CIF file for {COD_ID} already exist')
    return

  url = f"https://www.crystallography.net/cod/{COD_ID}.cif"
  response = requests.get(url)

  ##with open(f"{material_id}.cif", "w") as f:
  #    f.write(cif_string)
  if response.status_code == 200:
    print("Sucess")
    with open(path, 'w') as f:
        f.write(response.text)
  else:
    print('No CIF')

In [52]:
for material_id in df['COD_ID'].unique():
  extract_cif_from_COD(material_id)

1010942
CIF file for 1010942 already exist
1526720
CIF file for 1526720 already exist
1526777
CIF file for 1526777 already exist
8101224
CIF file for 8101224 already exist
7039788
CIF file for 7039788 already exist
1001842
CIF file for 1001842 already exist
2102087
CIF file for 2102087 already exist
1522043
CIF file for 1522043 already exist
2229871
CIF file for 2229871 already exist
7221013
CIF file for 7221013 already exist
1540895
CIF file for 1540895 already exist
1530317
CIF file for 1530317 already exist
1544432
CIF file for 1544432 already exist
1522041
CIF file for 1522041 already exist
1522039
CIF file for 1522039 already exist
2002850
CIF file for 2002850 already exist
1001022
CIF file for 1001022 already exist
2238958
CIF file for 2238958 already exist
2002197
CIF file for 2002197 already exist
2106523
CIF file for 2106523 already exist
sd_1045310
CIF file for sd_1045310 already exist
sd_0548135
CIF file for sd_0548135 already exist
1011128
CIF file for 1011128 already exist

In [53]:
for material_id in df['Materials Project ID'].unique():
  extract_CIF_from_MP(material_id)

ID:  mp-1245098
CIF file for mp-1245098 already exist
ID:  mp-541600
CIF file for mp-541600 already exist
Skipping invalid ID: -1
ID:  mp-3249
CIF file for mp-3249 already exist
ID:  mp-1228245
CIF file for mp-1228245 already exist
ID:  mp-1228150
CIF file for mp-1228150 already exist
ID:  mp-561133
CIF file for mp-561133 already exist
ID:  mp-3563
CIF file for mp-3563 already exist
ID:  mp-560692
CIF file for mp-560692 already exist
ID:  mp-3614
CIF file for mp-3614 already exist
ID:  mp-10347
CIF file for mp-10347 already exist
ID:  mp-1221096
CIF file for mp-1221096 already exist
ID:  mp-1104930
CIF file for mp-1104930 already exist
ID:  mp-990430
CIF file for mp-990430 already exist
ID:  mp-6259
CIF file for mp-6259 already exist
ID:  mp-7148
CIF file for mp-7148 already exist
ID:  mp-5308
CIF file for mp-5308 already exist
ID:  mp-3349
CIF file for mp-3349 already exist
ID:  mp-1204933
CIF file for mp-1204933 already exist
Skipping invalid ID: -2
ID:  mp-1220741
CIF file for mp-12

ValueError: Invalid identifier HLaTiO4_OсNH2, must be one of MPID, AlphaID, ThermoID, BatteryID, or XasSpectrumID.

##Valence electron discriptors

In [54]:
!pip install matminer
from matminer.featurizers.composition import ElementProperty
from matminer.featurizers.composition import ValenceOrbital
from pymatgen.core.composition import Composition


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 19.7 MB/s eta 0:00:00


In [55]:
df['composition_obj'] = df['Hill formula'].apply(Composition)
print("Null compositions",df['composition_obj'].isna().sum())
vo_feat = ValenceOrbital()
df = vo_feat.featurize_dataframe(df, col_id='composition_obj')
df = df.drop('composition_obj', axis=1)

Null compositions 0


/usr/local/lib/python3.12/dist-packages/matminer/featurizers/composition/orbital.py:115: UserWarning:

ValenceOrbital(impute_nan=False):
In a future release, impute_nan will be set to True by default.
                    This means that features that are missing or are NaNs for elements
                    from the data source will be replaced by the average of that value
                    over the available elements.
                    This avoids NaNs after featurization that are often replaced by
                    dataset-dependent averages.

/usr/local/lib/python3.12/dist-packages/matminer/utils/data.py:326: UserWarning:

MagpieData(impute_nan=False):
In a future release, impute_nan will be set to True by default.
                    This means that features that are missing or are NaNs for elements
                    from the data source will be replaced by the average of that value
                    over the available elements.
                    This avoids NaNs after f

ValenceOrbital:   0%|          | 0/512 [00:00<?, ?it/s]

##Mulliken electronegativity

In [56]:
from pymatgen.core.periodic_table import Element

def get_mulliken_en(element_symbol):
    el = Element(element_symbol)
    IE = el.ionization_energies[0]  # First ionization energy in eV
    EA = el.electron_affinity       # Electron affinity in eV

    if IE is None or EA is None:
        return None

    return (IE + EA) / 2


def calc_average_electronegativity(formula):
  # Example: For Fe2O3
  comp = Composition(formula)
  # Weighted mean electronegativity
  total_atoms = comp.num_atoms
  mean_en = sum(
    #comp[el] / total_atoms * Element(el).X
    comp[el] / total_atoms * get_mulliken_en(el)
    for el in comp.elements)
  return mean_en

In [57]:
df["Average Mulliken electronegativity"] = df["Hill formula"].apply(calc_average_electronegativity)

##Valence electron density

In [58]:
def split_formula(formula):
  output={}
  elements_with_indexes = formula.split()
  for el in elements_with_indexes:
    #match = re.match(r"([A-Za-z]+)(\d+)$", el)
    match = re.match(r"([A-Za-z]+)(\d+(?:\.\d+)?)$", el)
    if match:
      output[match.group(1)]=float(match.group(2))
    else:
      output[ el]=float(1)
  return output

def get_valence_electrons_number(hill_fomula):
  split = split_formula(hill_fomula)
  print(split)
  v = split.get("O")
  if v == None:
    return 0
  else:
    return 2*v

In [59]:
df["Valence electrons"] = df["Hill formula"].apply(get_valence_electrons_number)

{'O': 2.0, 'Ti': 1.0}
{'O': 2.0, 'Ti': 1.0}
{'La': 1.0, 'O': 7.0, 'Rb': 1.0, 'Ta': 2.0}
{'O': 7.0, 'Pr': 1.0, 'Rb': 1.0, 'Ta': 2.0}
{'Nd': 1.0, 'O': 7.0, 'Rb': 1.0, 'Ta': 2.0}
{'O': 7.0, 'Rb': 1.0, 'Sm': 1.0, 'Ta': 2.0}
{'O': 5.0, 'Ta': 2.0}
{'Nb': 2.0, 'O': 5.0}
{'La': 4.0, 'O': 12.0, 'Ti': 3.0}
{'Ca': 1.0, 'La': 4.0, 'O': 15.0, 'Ti': 4.0}
{'La': 4.0, 'O': 15.0, 'Sr': 1.0, 'Ti': 4.0}
{'Ba': 1.0, 'La': 4.0, 'O': 15.0, 'Ti': 4.0}
{'Ba': 3.0, 'La': 1.0, 'Nb': 3.0, 'O': 12.0}
{'Nb': 4.0, 'O': 15.0, 'Sr': 5.0}
{'Ba': 5.0, 'Nb': 4.0, 'O': 15.0}
{'La': 4.0, 'O': 12.0, 'Ti': 3.0}
{'Ca': 1.0, 'La': 4.0, 'O': 15.0, 'Ti': 4.0}
{'La': 4.0, 'O': 15.0, 'Sr': 1.0, 'Ti': 4.0}
{'Ca': 1.0, 'La': 4.0, 'O': 15.0, 'Ti': 4.0}
{'K': 4.0, 'Nb': 6.0, 'O': 17.0}
{'K': 4.0, 'Nb': 4.0, 'O': 17.0, 'Ta': 2.0}
{'K': 4.0, 'Nb': 3.0, 'O': 17.0, 'Ta': 3.0}
{'K': 4.0, 'Nb': 2.0, 'O': 17.0, 'Ta': 4.0}
{'K': 4.0, 'O': 17.0, 'Ta': 6.0}
{'Nb': 6.0, 'O': 17.0, 'Rb': 4.0}
{'Nb': 4.0, 'O': 17.0, 'Rb': 4.0, 'Ta': 2.0}
{'Nb': 3

##CIF files descriptors

In [60]:
from pymatgen.core.structure import Structure

def getCellVolume(ID):
  print("getCellVolume() ID:",ID)
  if(ID==-1):
    return 0
  file_path=f"/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/CIF/{ID}.cif"
  if os.path.exists(file_path):
    try:
      structure = Structure.from_file(file_path)
    except:
      print('ERROR: Invalid structure for ',ID)
      return 0
  else:
    return 0

  if(structure == None):
    return 0
  return structure.volume

In [61]:
def getCellZValue(ID):
  if(ID==-1):
    return 0
  file_path=f"/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/CIF/{ID}.cif"
  if os.path.exists(file_path):
    try:
      structure = Structure.from_file(file_path)
    except:
      print('ERROR: Invalid structure for ',ID)
      return 0
  else:
    return 0
  if(structure == None):
    return 0
  return structure.composition.num_atoms / structure.composition.reduced_composition.num_atoms

In [62]:
def abcExtractionFromMP(ID):
  if(ID==-1):
    return 0
  file_path=f"/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/CIF/{ID}.cif"
  if os.path.exists(file_path):
    try:
      structure = Structure.from_file(file_path)
    except:
      print('ERROR: Invalid structure for ',ID)
      return [0,0,0]
  else:
    return  [0,0,0]

  if(structure == None):
    return  [0,0,0]
  a = structure.lattice.a
  b = structure.lattice.b
  c = structure.lattice.c
  return [a,b,c]

In [63]:
df["Volume_MP"]=df["Materials Project ID"].apply(getCellVolume)
df["Z_MP"]=df["Materials Project ID"].apply(getCellZValue)
df['Valence Electrons Density_MP'] = (df['Valence electrons'] * df['Z_MP']) / df['Volume_MP']
df[['a_MP', 'b_MP', 'c_MP']] = df['Materials Project ID'].apply(abcExtractionFromMP).apply(pd.Series)

getCellVolume() ID: mp-1245098
getCellVolume() ID: mp-1245098
getCellVolume() ID: mp-541600
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: mp-3249
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: mp-1228245


/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: 24 fractional coordinates rounded to ideal values to avoid issues with finite precision.



getCellVolume() ID: mp-1228150
getCellVolume() ID: mp-561133
getCellVolume() ID: mp-3563


/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: 12 fractional coordinates rounded to ideal values to avoid issues with finite precision.



getCellVolume() ID: mp-3249
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: mp-1228245
getCellVolume() ID: mp-560692
getCellVolume() ID: mp-560692
getCellVolume() ID: mp-560692
getCellVolume() ID: mp-560692
getCellVolume() ID: mp-560692
getCellVolume() ID: mp-560692
getCellVolume() ID: mp-560692
getCellVolume() ID: mp-560692
getCellVolume() ID: mp-560692
getCellVolume() ID: mp-560692
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: mp-3614
getCellVolume() ID: mp-10347
getCellVolume() ID: mp-1221096
getCellVolume() ID: -1
getCellVolume() ID: mp-10347
getCellVolume() ID: mp-1221096
getCellVolume() ID: mp-1104930
getCellVolume() ID: mp-1104930
getCellVolume() ID: mp-1104930
getCellVolume() ID: mp-1104930
getCellVolume() ID: mp-1104930
getCellVolume() ID: mp-1104930
getCellVolume() ID: mp-1104930
getCellVolume() ID: mp-1104930
getCellVolume() ID: mp-1104930
getCellVolume() ID: mp-1104930
getCellVolume() ID: mp

/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:1314: UserWarning:

Missing elements K from PMG structure composition



getCellVolume() ID: HLaTiO4_EtNH2
getCellVolume() ID: HLaTiO4_PrNH2
getCellVolume() ID: HLaTiO4_BuNH2
getCellVolume() ID: HLaTiO4_HxNH2
getCellVolume() ID: HLaTiO4_OсNH2
getCellVolume() ID: HLaTiO4_MeOH
getCellVolume() ID: HLaTiO4_EtOH
getCellVolume() ID: HLaTiO4_PrOH
getCellVolume() ID: HLaTiO4_BuOH
getCellVolume() ID: HLaTiO4_HxOH
getCellVolume() ID: HLaTiO4_DcOH
getCellVolume() ID: mp-1104930
getCellVolume() ID: HLaTiO4_MeNH2
getCellVolume() ID: HLaTiO4_EtNH2
getCellVolume() ID: HLaTiO4_PrNH2
getCellVolume() ID: HLaTiO4_BuNH2
getCellVolume() ID: HLaTiO4_HxNH2
getCellVolume() ID: HLaTiO4_OсNH2
getCellVolume() ID: HLaTiO4_MeOH
getCellVolume() ID: HLaTiO4_EtOH
getCellVolume() ID: HLaTiO4_PrOH
getCellVolume() ID: HLaTiO4_BuOH
getCellVolume() ID: HLaTiO4_HxOH
getCellVolume() ID: HLaTiO4_DcOH
getCellVolume() ID: mp-1104930
getCellVolume() ID: HLaTiO4_MeNH2
getCellVolume() ID: HLaTiO4_EtNH2
getCellVolume() ID: HLaTiO4_PrNH2
getCellVolume() ID: HLaTiO4_BuNH2
getCellVolume() ID: HLaTiO4_HxNH

/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:286: UserWarning:

Possible issue in CIF file at line: 1

/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:1314: UserWarning:

Cannot determine chemical composition from CIF! _chemical_formula_sum is an invalid formula!



getCellVolume() ID: HCa2Ta3O10_EtOH
getCellVolume() ID: HCa2Ta3O10_PrOH
getCellVolume() ID: HCa2Ta3O10_BuOH
getCellVolume() ID: HCa2Ta3O10_HxOH
getCellVolume() ID: HCa2Ta3O10_OcOH
getCellVolume() ID: mp-557195
getCellVolume() ID: mp-10347
getCellVolume() ID: HCa2Ta3O10_MeOH
getCellVolume() ID: HCa2Ta3O10_EtOH
getCellVolume() ID: HCa2Ta3O10_PrOH
getCellVolume() ID: HCa2Ta3O10_BuOH
getCellVolume() ID: HCa2Ta3O10_HxOH
getCellVolume() ID: HCa2Ta3O10_OcOH
getCellVolume() ID: mp-6548
getCellVolume() ID: mp-6548
getCellVolume() ID: mp-6548
getCellVolume() ID: mp-6548
getCellVolume() ID: mp-1245098
getCellVolume() ID: mp-6548
getCellVolume() ID: K2La2Ti3O10_MeNH2
getCellVolume() ID: K2La2Ti3O10_EtNH2
getCellVolume() ID: K2La2Ti3O10_PrNH2
getCellVolume() ID: K2La2Ti3O10_BuNH2
getCellVolume() ID: K2La2Ti3O10_HxNH2
getCellVolume() ID: K2La2Ti3O10_OcNH2
getCellVolume() ID: K2La2Ti3O10_MeOH
getCellVolume() ID: K2La2Ti3O10_EtOH
getCellVolume() ID: K2La2Ti3O10_PrOH
getCellVolume() ID: K2La2Ti3O10_BuO

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: 24 fractional coordinates rounded to ideal values to avoid issues with finite precision.

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: 12 fractional coordinates rounded to ideal values to avoid issues with finite precision.

/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:1314: UserWarning:

Missing elements K from PMG structure composition

/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:286: UserWarning:

Possible issue in CIF file at line: 1

/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:1314: UserWarning:

Cannot determine chemical composition from CIF! _chemical_formula_sum is an invalid formula!

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: 24 fractional coordinates

In [64]:
df["Volume_COD"]=df["COD_ID"].apply(getCellVolume)
df["Z_COD"]=df["COD_ID"].apply(getCellZValue)
df['Valence Electrons Density_COD'] = (df['Valence electrons'] * df['Z_COD']) / df['Volume_COD']
df[['a_COD', 'b_COD', 'c_COD']] = df['COD_ID'].apply(abcExtractionFromMP).apply(pd.Series)

getCellVolume() ID: 1010942
getCellVolume() ID: 1010942
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: 1526720
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: 1526777


/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: 8 fractional coordinates rounded to ideal values to avoid issues with finite precision.



getCellVolume() ID: -1
getCellVolume() ID: 8101224


/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: 3 fractional coordinates rounded to ideal values to avoid issues with finite precision.



getCellVolume() ID: 7039788


...

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

No structure parsed for section 1 in CIF.
'_atom_site_label'

/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:1029: UserWarning:

No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.

/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:1314: UserWarning:

Cannot determine chemical composition from CIF! 'NoneType' object is not iterable

/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:1029: UserWarning:

No _symmetry_equiv_pos_as_xyz type key found. Defaulting to P1.

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: Pauling file corrections applied.
Pauling file corrections applied.
No structure parsed for section 1 in CIF.
'_atom_site_label'
No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.
No _symmetry_

getCellVolume() ID: sd_0548135


/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: Pauling file corrections applied.
No structure parsed for section 1 in CIF.
'_atom_site_label'
No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.
No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.
No _symmetry_equiv_pos_as_xyz type key found. Defaulting to P1.



getCellVolume() ID: -1
getCellVolume() ID: sd_1045310
getCellVolume() ID: sd_0548135
getCellVolume() ID: -1
getCellVolume() ID: sd_1045310
getCellVolume() ID: sd_0548135
getCellVolume() ID: -1
getCellVolume() ID: 1011128
getCellVolume() ID: 1001022
getCellVolume() ID: 1001022
getCellVolume() ID: 2106523
getCellVolume() ID: 1001842
getCellVolume() ID: 1001842
getCellVolume() ID: 1001842
getCellVolume() ID: 1001842
getCellVolume() ID: 1001842
getCellVolume() ID: 1001842
getCellVolume() ID: 1001842
getCellVolume() ID: 1001842
getCellVolume() ID: 1001842
getCellVolume() ID: 1001842
getCellVolume() ID: 1001842
getCellVolume() ID: 1001842
getCellVolume() ID: 1001842
getCellVolume() ID: 1001842
getCellVolume() ID: 1001842
getCellVolume() ID: 1001842
getCellVolume() ID: 1001842
getCellVolume() ID: 1001842
getCellVolume() ID: 7039788
getCellVolume() ID: 7039788
getCellVolume() ID: 7039788
getCellVolume() ID: 7039788
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: 1521061
getCe

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: 8 fractional coordinates rounded to ideal values to avoid issues with finite precision.

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: 3 fractional coordinates rounded to ideal values to avoid issues with finite precision.

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: 6 fractional coordinates rounded to ideal values to avoid issues with finite precision.

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

No structure parsed for section 1 in CIF.
'_atom_site_label'

/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:1029: UserWarning:

No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.

/usr/local/lib/python3.12/dist

In [65]:
df["Volume_Springer"]=df["Springer_ID"].apply(getCellVolume)
df["Z_Springer"]=df["Springer_ID"].apply(getCellZValue)
df['Valence Electrons Density_Springer'] = (df['Valence electrons'] * df['Z_Springer']) / df['Volume_Springer']
df[['a_Springer', 'b_Springer', 'c_Springer']] = df['Springer_ID'].apply(abcExtractionFromMP).apply(pd.Series)

getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: sd_1050391


/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

No structure parsed for section 1 in CIF.
'_atom_site_label'

/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:1029: UserWarning:

No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.

/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:1314: UserWarning:

Cannot determine chemical composition from CIF! 'NoneType' object is not iterable

/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:1029: UserWarning:

No _symmetry_equiv_pos_as_xyz type key found. Defaulting to P1.

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: No structure parsed for section 1 in CIF.
'_atom_site_label'
No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.
No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_gro

getCellVolume() ID: sd_1050391
getCellVolume() ID: sd_1050391
getCellVolume() ID: sd_1050391
getCellVolume() ID: sd_1232960
getCellVolume() ID: sd_0314261
getCellVolume() ID: -1
getCellVolume() ID: sd_0306542


/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: Pauling file corrections applied.
10 fractional coordinates rounded to ideal values to avoid issues with finite precision.
No structure parsed for section 1 in CIF.
'_atom_site_label'
No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.
No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.
No _symmetry_equiv_pos_as_xyz type key found. Defaulting to P1.



getCellVolume() ID: sd_0306445
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: sd_0306542
getCellVolume() ID: sd_0306445
getCellVolume() ID: sd_0306542
getCellVolume() ID: sd_0376709
getCellVolume() ID: sd_0376709
getCellVolume() ID: sd_0376709
getCellVolume() ID: sd_0376709
getCellVolume() ID: sd_0376709
getCellVolume() ID: sd_0376709
getCellVolume() ID: sd_0376709
getCellVolume() ID: sd_0376709
getCellVolume() ID: sd_0376709
getCellVolume() ID: sd_0376709
getCellVolume() ID: sd_0314261
getCellVolume() ID: sd_0308503
getCellVolume() ID: sd_1127671
getCellVolume() ID: sd_1232960
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCellVolume() ID: -1
getCe

...

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: 4 fractional coordinates rounded to ideal values to avoid issues with finite precision.
No structure parsed for section 1 in CIF.
'_atom_site_label'
No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.
No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.
No _symmetry_equiv_pos_as_xyz type key found. Defaulting to P1.

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

No structure parsed for section 1 in CIF.
'_atom_site_label'

/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:1029: UserWarning:

No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.

/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:1314: UserWarning:

Cannot determine chemical composition from CIF! 'NoneTyp

ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1925412


/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:286: UserWarning:

Possible issue in CIF file at line: ?

/usr/local/lib/python3.12/dist-packages/pymatgen/io/cif.py:1044: UserWarning:

t parsed as T

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

No structure parsed for section 3 in CIF.
could not convert string to float: 't'

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

No structure parsed for section 4 in CIF.
could not convert string to float: 't'

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: No structure parsed for section 1 in CIF.
'_atom_site_label'
No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.
No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.
t parsed as T
No structure parsed for section 3 in CIF.
could not c

ERROR: Invalid structure for  sd_1210175
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1241787
ERROR: Invalid structure for  sd_1210175
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1241787
ERROR: Invalid structure for  sd_1210175
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1241787
ERROR: Invalid structure for  sd_1210175
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1241787
ERROR: Invalid structure for  sd_1210175
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1241787
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1925412
ERROR: Invalid structure for  sd_1241787
ERROR: Invalid structure for  sd_1210175
ERROR: Invalid structure for  sd_1210175
ERROR: Invalid s

/usr/local/lib/python3.12/dist-packages/pymatgen/core/structure.py:3109: UserWarning:

Issues encountered while parsing CIF: 4 fractional coordinates rounded to ideal values to avoid issues with finite precision.
No structure parsed for section 1 in CIF.
'_atom_site_label'
No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.
No _symmetry_equiv_pos_as_xyz type key found. Spacegroup from _symmetry_space_group_name_H-M used.
No _symmetry_equiv_pos_as_xyz type key found. Defaulting to P1.



In [66]:
df["Volume_manual"]=df["a, A"]*df["b, A"]*df["c, A"]
df['Valence Electrons Density_manual'] = (df['Valence electrons'] * df['Z']) / df['Volume_manual']

##Oxygen concentration

In [67]:
def count_oxigen(formula):
  #match = re.search(r'O(\d+)', hill_formula)
  #if match:
  #  number = int(match.group(1))
  #  print(number)
  #  return number
  #else:
  #  print("No match found")
  if formula is None:
    return 0
  print(formula)
  comp = Composition(formula)

  # Get number of oxygen atoms
  oxygen_count = comp.get_el_amt_dict().get("O", 0)
  print(oxygen_count)
  return oxygen_count

In [68]:
df['Oxygen_count']=df['Hill formula'].apply(count_oxigen)

O2 Ti
2.0
O2 Ti
2.0
La O7 Rb Ta2
7.0
O7 Pr Rb Ta2
7.0
Nd O7 Rb Ta2
7.0
O7 Rb Sm Ta2
7.0
O5 Ta2
5.0
Nb2 O5
5.0
La4 O12 Ti3
12.0
Ca La4 O15 Ti4
15.0
La4 O15 Sr Ti4
15.0
Ba La4 O15 Ti4
15.0
Ba3 La Nb3 O12
12.0
Nb4 O15 Sr5
15.0
Ba5 Nb4 O15
15.0
La4 O12 Ti3
12.0
Ca La4 O15 Ti4
15.0
La4 O15 Sr Ti4
15.0
Ca La4 O15 Ti4
15.0
K4 Nb6 O17
17.0
K4 Nb4 O17 Ta2
17.0
K4 Nb3 O17 Ta3
17.0
K4 Nb2 O17 Ta4
17.0
K4 O17 Ta6
17.0
Nb6 O17 Rb4
17.0
Nb4 O17 Rb4 Ta2
17.0
Nb3 O17 Rb4 Ta3
17.0
Nb2 O17 Rb4 Ta4
17.0
O17 Rb4 Ta6
17.0
Nb2 O5
5.0
K Nb O3
3.0
K Nb3 O8
8.0
O5 Ta2
5.0
K O3 Ta
3.0
Ca2 Cs O10 Ta3
10.0
Ca2 Cs H Na O10 Ta3
10.0
C6 H16 Ca2 N O10 Ta3
10.0
Ca2 Cs O10 Ta3
10.0
Ca2 Cs H Na O10 Ta3
10.0
K La O4 Ti
4.0
K La O4 Ti Zr
4.0
K La O4 Ti Zr
4.0
K La O4 Ti Zr
4.0
K La O4 Ti Zr
4.0
K La O4 Zr
4.0
K La O4 Ti
4.0
K La O4 Ti Zr
4.0
K La O4 Ti Zr
4.0
K La O4 Ti Zr
4.0
K La O4 Ti Zr
4.0
K La O4 Zr
4.0
H2 O7 Sr Ta2
7.0
Li2 O7 Sr Ta2
7.0
K2 O7 Sr Ta2
7.0
O7 Rb2 Sr Ta2
7.0
K O3 Ta
3.0
H2 O7 Sr Ta2
7.0
K2 O7 Sr Ta2
7.

In [69]:
df['Oxygen_concentration_manual']=df['Z']*df['Oxygen_count']/df['Volume_manual']
df['Oxygen_concentration_MP']=df['Z_MP']*df['Oxygen_count']/df['Volume_MP']
df['Oxygen_concentration_COD']=df['Z_COD']*df['Oxygen_count']/df['Volume_COD']
df['Oxygen_concentration_Springer']=df['Z_Springer']*df['Oxygen_count']/df['Volume_Springer']

##Packing fraction


In [70]:
ionic_radii={"H+":0.02}
difficult_compunds_oxidation_states={
    "Ca Cs Na O10 Ta3":{"Ca":2,"Cs":1,"Na":1,"O":-2,"Ta":5},
    "Fe K2 La2 O10 Ti2":{"Fe":3, "K": 1,"La":3,"O":-2,"Ti":4},
    'Ca2 H Nb O10 Ta':{"Ca":2,"H":1,"Nb": 5,"Ta":5,"O":-2},
    'K2 O10 Sr Ta3':{"K":1,"O":-2,"Sr":2,"Ta":5},
    'H Nb O10 Sr2 Ta':{"H":1,"Nb":5,"O":-2,"Sr":2,"Ta":5},
}

In [71]:
from pymatgen.core import Structure
from pymatgen.analysis.local_env import ValenceIonicRadiusEvaluator
def get_packing_fraction_from_cif(mp_id):
    print(mp_id)
    cif_folder = "/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/CIF/"
    cif_path = os.path.join(cif_folder, f"{mp_id}.cif")

    if not os.path.isfile(cif_path):
        return np.nan  # Return NaN if file doesn't exist
    try:
      structure = Structure.from_file(cif_path)
    except:
      return 0
    comp = structure.composition
    #comp_oxi = comp.oxidation_state_guesses()
    #print(comp_oxi)


    # Extract radius for each site
    site_radii = []

    try:
      print('Calculating oxidation states')
      structure.add_oxidation_state_by_guess()
    except:
      print('Error')
      return


    for site in structure:
        #el = Element(site.species_string)
        try:
          specie = site.specie
        except AttributeError:
          return 0
        print(site.specie.ionic_radii)
        print(site.specie.oxi_state)
        print('Specie ionic radii ',site.specie.ionic_radii)
        print('Site oxi satte ',site.specie.oxi_state)
        rad =site.specie.ionic_radii.get(site.specie.oxi_state)
        print('got site')
        if(rad is not None):
          site_radii.append(rad)

    # Compute atomic volume per site
    atom_volumes = [(4/3) * np.pi * r**3 for r in site_radii]
    total_atom_volume = sum(atom_volumes)

    # Compute packing fraction
    packing_fraction = total_atom_volume / structure.volume
    print('OK')
    return packing_fraction
    #except:
        #return np.nan  # Return NaN if anything fails (e.g., missing radii)

In [72]:
from pymatgen.core import Structure
from pymatgen.analysis.local_env import ValenceIonicRadiusEvaluator
from pymatgen.core.periodic_table import Species

unresolved_compunds = []

def get_packing_fraction_from_formula_and_cell_volume(hill_formula, V, Z):
    if V==0 or np.isnan(V) or Z==0 or np.isnan(Z):
        return np.nan
    print("Enter!")
    print(hill_formula)
    print(V)
    print(Z)
    comp = Composition(hill_formula)
    oxi_guesses = comp.oxi_state_guesses()
    if(len(oxi_guesses)==0):
      oxi_guesses = difficult_compunds_oxidation_states.get(hill_formula)
      if(oxi_guesses==None):
        unresolved_compunds.append(hill_formula)
        return np.nan
    else:
      oxi_guesses = oxi_guesses[0]

    V_ions_formula=0
    for el, amt in comp.items():
        symbol = el.symbol
        el_oxidation_state = oxi_guesses[symbol]
        #element = Element(el)
        print("Element:", symbol)
        print("Element ox state:", el_oxidation_state)
        specie = Species(symbol,el_oxidation_state)
        r = specie.ionic_radius
        if(r==np.nan or r==None):
          ion_formula = symbol
          if(el_oxidation_state!=0 and el_oxidation_state!=1 and el_oxidation_state!=-1):
             ion_formula =  ion_formula+str(el_oxidation_state)
          if(el_oxidation_state>0):
            ion_formula =  ion_formula+str("+")
          if(el_oxidation_state<0):
            ion_formula =  ion_formula+str("-")
          print("Local ionic radii table request for ",ion_formula)
          r= ionic_radii.get(ion_formula)
          if(r==None):
            r=0
        print("r: ", r)
        V_ion = (4/3) * np.pi * r**3
        V_ions_formula += (V_ion*amt)
        print(el," ",amt)

    packing_fraction= Z*V_ions_formula/V
    return packing_fraction
    print("Output!")

In [73]:
#fr = get_packing_fraction_from_formula_and_cell_volume("H2 La2 Ti3 O10",390.7,2)
#print("Fraction: ", fr)

#fr = get_packing_fraction_from_formula_and_cell_volume("O2 Ti",1183,30)
#print("Fraction: ", fr)

fr = get_packing_fraction_from_formula_and_cell_volume("Ca Cs Na O10 Ta3",228,1)
print("Fraction: ", fr)

Enter!
Ca Cs Na O10 Ta3
228
1
Element: Ca
Element ox state: 2
r:  1.14 ang
Ca   1.0
Element: Cs
Element ox state: 1
r:  1.81 ang
Cs   1.0
Element: Na
Element ox state: 1
r:  1.16 ang
Na   1.0
Element: O
Element ox state: -2
r:  1.26 ang
O   10.0
Element: Ta
Element ox state: 5
r:  0.78 ang
Ta   3.0
Fraction:  0.5584979567677797


In [74]:
#df["MP_packing_fraction"] = df["Materials Project ID"].apply(get_packing_fraction_from_cif)
#df["COD_packing_fraction"] = df["COD_ID"].apply(get_packing_fraction_from_cif)
#df["Springer_packing_fraction"] = df["Springer_ID"].apply(get_packing_fraction_from_cif)

df["MP_packing_fraction"] = df.apply(lambda row: get_packing_fraction_from_formula_and_cell_volume(row["Hill formula"], row["Volume_MP"], row["Z_MP"]), axis=1)
df["COD_packing_fraction"] = df.apply(lambda row: get_packing_fraction_from_formula_and_cell_volume(row["Hill formula"], row["Volume_COD"], row["Z_COD"]), axis=1)
df["Springer_packing_fraction"] = df.apply(lambda row: get_packing_fraction_from_formula_and_cell_volume(row["Hill formula"], row["Volume_Springer"], row["Z_Springer"]), axis=1)
df["Manual_packing_fraction"] = df.apply(lambda row: get_packing_fraction_from_formula_and_cell_volume(row["Hill formula"], row["Volume_manual"], row["Z"]), axis=1)

Enter!
O2 Ti
1182.7412601085477
30.0
Element: O
Element ox state: -2.0
r:  1.26 ang
O   2.0
Element: Ti
Element ox state: 4.0
r:  0.745 ang
Ti   1.0
Enter!
O2 Ti
1182.7412601085477
30.0
Element: O
Element ox state: -2.0
r:  1.26 ang
O   2.0
Element: Ti
Element ox state: 4.0
r:  0.745 ang
Ti   1.0
Enter!
La O7 Rb Ta2
173.01142785646587
1.0
Element: La
Element ox state: 3.0
r:  1.172 ang
La   1.0
Element: O
Element ox state: -2.0
r:  1.26 ang
O   7.0
Element: Rb
Element ox state: 1.0
r:  1.66 ang
Rb   1.0
Element: Ta
Element ox state: 5.0
r:  0.78 ang
Ta   2.0
Enter!
La4 O12 Ti3
238.90888175434102
1.0
Element: La
Element ox state: 3.0
r:  1.172 ang
La   4.0
Element: O
Element ox state: -2.0
r:  1.26 ang
O   12.0
Element: Ti
Element ox state: 4.0
r:  0.745 ang
Ti   3.0
Enter!
Ba La4 O15 Ti4
623.8123076220189
2.0
Element: Ba
Element ox state: 2.0
r:  1.49 ang
Ba   1.0
Element: La
Element ox state: 3.0
r:  1.172 ang
La   4.0
Element: O
Element ox state: -2.0
r:  1.26 ang
O   15.0
Element: T

/tmp/ipython-input-1623332607.py:32: UserWarning:

No ionic radius for H+!

/tmp/ipython-input-1623332607.py:32: UserWarning:

No ionic radius for Ta4.33+!

/tmp/ipython-input-1623332607.py:32: UserWarning:

No ionic radius for Zr2+!

/tmp/ipython-input-1623332607.py:32: UserWarning:

No ionic radius for N+!

/tmp/ipython-input-1623332607.py:32: UserWarning:

No ionic radius for Ta4.67+!



Element: K
Element ox state: 1.0
r:  1.52 ang
K   2.0
Element: La
Element ox state: 3.0
r:  1.172 ang
La   2.0
Element: O
Element ox state: -2.0
r:  1.26 ang
O   9.0
Element: Ti
Element ox state: 4.0
r:  0.745 ang
Ti   2.0
Element: Zn
Element ox state: 2.0
r:  0.88 ang
Zn   1.0
Enter!
K2 La2 O9 Ti2 Zn
229.90248606119223
1.0
Element: K
Element ox state: 1.0
r:  1.52 ang
K   2.0
Element: La
Element ox state: 3.0
r:  1.172 ang
La   2.0
Element: O
Element ox state: -2.0
r:  1.26 ang
O   9.0
Element: Ti
Element ox state: 4.0
r:  0.745 ang
Ti   2.0
Element: Zn
Element ox state: 2.0
r:  0.88 ang
Zn   1.0
Enter!
Ca2 Cs O10 Ta3
242.12790249231253
1.0
Element: Ca
Element ox state: 2.0
r:  1.14 ang
Ca   2.0
Element: Cs
Element ox state: 1.0
r:  1.81 ang
Cs   1.0
Element: O
Element ox state: -2.0
r:  1.26 ang
O   10.0
Element: Ta
Element ox state: 5.0
r:  0.78 ang
Ta   3.0
Enter!
Ca2 Cs N O10 Ta3
242.12790249231253
1.0
Element: Ca
Element ox state: 2.0
r:  1.14 ang
Ca   2.0
Element: Cs
Element ox 

/tmp/ipython-input-1623332607.py:32: UserWarning:

No ionic radius for C2-!

/tmp/ipython-input-1623332607.py:32: UserWarning:

No ionic radius for C2.33-!



Element: C
Element ox state: -2.0
Local ionic radii table request for  C-2.0-
r:  0
C   6.0
Element: H
Element ox state: 1.0
Local ionic radii table request for  H+
r:  0.02
H   16.0
Element: La
Element ox state: 3.0
r:  1.172 ang
La   1.0
Element: N
Element ox state: -3.0
r:  1.32 ang
N   1.0
Element: O
Element ox state: -2.0
r:  1.26 ang
O   4.0
Element: Ti
Element ox state: 4.0
r:  0.745 ang
Ti   1.0
Enter!
C H5 La O5 Ti
249.4153894545576
2.0
Element: C
Element ox state: -2.0
Local ionic radii table request for  C-2.0-
r:  0
C   1.0
Element: H
Element ox state: 1.0
Local ionic radii table request for  H+
r:  0.02
H   5.0
Element: La
Element ox state: 3.0
r:  1.172 ang
La   1.0
Element: O
Element ox state: -2.0
r:  1.26 ang
O   5.0
Element: Ti
Element ox state: 4.0
r:  0.745 ang
Ti   1.0
Enter!
C2 H7 La O5 Ti
279.3751959956456
2.0
Element: C
Element ox state: -2.0
Local ionic radii table request for  C-2.0-
r:  0
C   2.0
Element: H
Element ox state: 1.0
Local ionic radii table reques

...

/tmp/ipython-input-1623332607.py:32: UserWarning:

No ionic radius for C2.33-!

/tmp/ipython-input-1623332607.py:32: UserWarning:

No ionic radius for N+!



Element: C
Element ox state: -2.0
Local ionic radii table request for  C-2.0-
r:  0
C   10.0
Element: H
Element ox state: 1.0
Local ionic radii table request for  H+
r:  0.02
H   23.0
Element: La
Element ox state: 3.0
r:  1.172 ang
La   1.0
Element: O
Element ox state: -2.0
r:  1.26 ang
O   5.0
Element: Ti
Element ox state: 4.0
r:  0.745 ang
Ti   1.0
Enter!
H Nd O4 Ti
163.86304
2.0
Element: H
Element ox state: 1.0
Local ionic radii table request for  H+
r:  0.02
H   1.0
Element: Nd
Element ox state: 3.0
r:  1.123 ang
Nd   1.0
Element: O
Element ox state: -2.0
r:  1.26 ang
O   4.0
Element: Ti
Element ox state: 4.0
r:  0.745 ang
Ti   1.0
Enter!
C H6 N Nd O4 Ti
462.9895600000001
4.0
Element: C
Element ox state: -2.0
Local ionic radii table request for  C-2.0-
r:  0
C   1.0
Element: H
Element ox state: 1.0
Local ionic radii table request for  H+
r:  0.02
H   6.0
Element: N
Element ox state: -3.0
r:  1.32 ang
N   1.0
Element: Nd
Element ox state: 3.0
r:  1.123 ang
Nd   1.0
Element: O
Elemen

/tmp/ipython-input-1623332607.py:32: UserWarning:

No ionic radius for C1.33-!



Element: C
Element ox state: -2.0
Local ionic radii table request for  C-2.0-
r:  0
C   8.0
Element: H
Element ox state: 1.0
Local ionic radii table request for  H+
r:  0.02
H   19.0
Element: Ca
Element ox state: 2.0
r:  1.14 ang
Ca   2.0
Element: Nb
Element ox state: 5.0
r:  0.78 ang
Nb   3.0
Element: O
Element ox state: -2.0
r:  1.26 ang
O   11.0
Enter!
La2 O10 Rb2 Ti3
462.1094215
2.0
Element: La
Element ox state: 3.0
r:  1.172 ang
La   2.0
Element: O
Element ox state: -2.0
r:  1.26 ang
O   10.0
Element: Rb
Element ox state: 1.0
r:  1.66 ang
Rb   2.0
Element: Ti
Element ox state: 4.0
r:  0.745 ang
Ti   3.0
Enter!
H La2 O10 Rb Ti3
479.52691200000004
2.0
Element: H
Element ox state: 1.0
Local ionic radii table request for  H+
r:  0.02
H   1.0
Element: La
Element ox state: 3.0
r:  1.172 ang
La   2.0
Element: O
Element ox state: -2.0
r:  1.26 ang
O   10.0
Element: Rb
Element ox state: 1.0
r:  1.66 ang
Rb   1.0
Element: Ti
Element ox state: 4.0
r:  0.745 ang
Ti   3.0
Enter!
H La2 O10 Rb T

/tmp/ipython-input-1623332607.py:32: UserWarning:

No default ionic radius for Mn4+. Using ls data.



In [75]:
print(unresolved_compunds)

['Ca2 K Nb2 O10 Rh', 'Ca2 H Nb2 O10 Rh', 'Ca2 H Nb2 O10 Rh', 'Ca2 H Nb2 O10 Rh', 'Ca2 H Nb2 O10 Rh', 'Ca2 H Nb2 O10 Rh', 'H2 K La2 O11 Ti3', 'H2 K La2 O11 Ti3', 'Ca2 K Nb2 O10 Rh', 'Ca2 H Nb2 O10 Rh', 'Ca2 H Nb2 O10 Rh', 'Ca2 H Nb2 O10 Rh', 'Ca2 H Nb2 O10 Rh', 'Ca2 H Nb2 O10 Rh', 'H2 K La2 O11 Ti3', 'H2 K La2 O11 Ti3']


##Averaging


In [76]:
def averaging_valence_electron_density(MP,COD,Springer):
  sum=0;
  count=0;
  print(MP)
  print(COD)
  print(Springer)
  if(pd.notna(MP)):
    sum+=MP
    count+=1
  if(pd.notna(COD)):
    sum+=COD
    count+=1
  if(pd.notna(Springer)):
    sum+=Springer
    count+=1
  print('Count: ',count)
  if count==0:
    return 0
  return sum/count


def averaging_4(MP,COD,Springer,manual):
  sum=0;
  count=0;
  print(MP)
  print(COD)
  print(Springer)
  print(manual)
  if(pd.notna(MP)):
    sum+=MP
    count+=1
  if(pd.notna(COD)):
    sum+=COD
    count+=1
  if(pd.notna(Springer)):
    sum+=Springer
    count+=1
  if(pd.notna(manual)):
    sum+=manual
    count+=1
  print('Count: ',count)
  if count==0:
    print("No value")
    return np.nan
  return sum/count

In [77]:
df['Valence Electrons Density avg'] = df.apply(lambda x: averaging_4(x['Valence Electrons Density_MP'], x['Valence Electrons Density_COD'],x['Valence Electrons Density_Springer'],x['Valence Electrons Density_manual']), axis=1)
df['Oxygen_concentration avg'] = df.apply(lambda x: averaging_4(x['Oxygen_concentration_MP'], x['Oxygen_concentration_COD'],x['Oxygen_concentration_Springer'],x['Oxygen_concentration_manual']), axis=1)
df['Packing fraction avg'] = df.apply(lambda x: averaging_4(x['MP_packing_fraction'], x['COD_packing_fraction'],x['Springer_packing_fraction'],x['Manual_packing_fraction']), axis=1)

Streaming output truncated to the last 5000 lines.
nan
Count:  3
0.03848232562392471
0.04048050183801514
0.03896164466727959
nan
Count:  3
nan
nan
0.051783465773676096
nan
Count:  1
nan
nan
0.046041274653656275
nan
Count:  1
nan
nan
0.04437652156580326
nan
Count:  1
nan
nan
0.060160196130029625
nan
Count:  1
0.04705541337351039
0.04728874617677464
nan
nan
Count:  2
0.041300485805503134
0.04386526126988772
nan
0.04421820641315219
Count:  3
0.043707319454494645
nan
nan
0.04636770255823598
Count:  2
nan
nan
nan
0.046855783637796355
Count:  1
0.041300485805503134
0.04386526126988772
nan
0.04421820641315219
Count:  3
0.043707319454494645
nan
nan
0.04636770255823598
Count:  2
0.03925429845533417
0.040246260329256864
nan
nan
Count:  2
0.03925429845533417
0.040246260329256864
nan
nan
Count:  2
0.03925429845533417
0.040246260329256864
nan
nan
Count:  2
0.03925429845533417
0.040246260329256864
nan
nan
Count:  2
0.03925429845533417
0.040246260329256864
nan
nan
Count:  2
0.03925429845533417
0.0402

In [78]:
df['Log_rate'] = np.log(df['Rate, umol/(g*h)'])
df['Log_rate'].replace([np.inf, -np.inf], np.nan, inplace=True)
df = df.dropna(subset=["Log_rate"])
df['Log_rate'].value_counts()

/usr/local/lib/python3.12/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning:

divide by zero encountered in log

/tmp/ipython-input-830342094.py:2: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





,count
Log_rate,
1.609438,8
1.791759,7
3.784190,6
6.253829,5
6.173786,5
...,...
4.158883,1
5.768321,1
2.830858,1


In [79]:
#raise SystemExit("Stopping execution")
print(df.shape[0])
df = df[df["Sac Agent 1"].isna()]
df = df[df["Sac agent 2"].isna()]
print(df.shape[0])

494
478


EDA

In [80]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 478 entries, 19 to 1088
Data columns (total 75 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Perovskite                          478 non-null    object 
 1   Interlayer space composition        3 non-null      object 
 2   Bandgap, eV                         478 non-null    float64
 3   Materials Project ID                478 non-null    object 
 4   COD_ID                              478 non-null    object 
 5   Springer_ID                         478 non-null    object 
 6   Z                                   342 non-null    float64
 7   a, A                                343 non-null    float64
 8   b, A                                342 non-null    float64
 9   c, A                                342 non-null    float64
 10  Symmetry group                      221 non-null    object 
 11  d,A                                 390 non-null

In [81]:
df.columns

Index(['Perovskite', 'Interlayer space composition', 'Bandgap, eV',
       'Materials Project ID', 'COD_ID', 'Springer_ID', 'Z', 'a, A', 'b, A',
       'c, A', 'Symmetry group', 'd,A', 'Number of octahedrons on a layer',
       'Prep Method', 'CalcT(K)', 'Calc time (h)', 'Nitrogen',
       'Promotion method', 'Promoter', 'Promoter, w%', 'Surface area, m2/g',
       'CatW, g/L', 'Alcohol, %', 'Sac Agent 1', 'Sac agent 2', 'Light type',
       'Power, W', 'Wave length min, nm', 'Cell material', 'Temperature, K',
       'Rate, umol/(g*h)', 'Hill formula', 'avg s valence electrons',
       'avg p valence electrons', 'avg d valence electrons',
       'avg f valence electrons', 'frac s valence electrons',
       'frac p valence electrons', 'frac d valence electrons',
       'frac f valence electrons', 'Average Mulliken electronegativity',
       'Valence electrons', 'Volume_MP', 'Z_MP',
       'Valence Electrons Density_MP', 'a_MP', 'b_MP', 'c_MP', 'Volume_COD',
       'Z_COD', 'Valence Elec

In [83]:
columns_EDA = ['Perovskite','Bandgap, eV',
       'Symmetry group', 'd,A', 'Number of octahedrons on a layer',
       'Prep Method', 'CalcT(K)', 'Calc time (h)', 'Nitrogen',
       'Promotion method', 'Promoter', 'Promoter, w%', 'Surface area, m2/g',
       'CatW, g/L', 'Alcohol, %', 'Light type',
       'Power, W', 'Wave length min, nm', 'Cell material', 'Temperature, K',
       'Rate, umol/(g*h)',
       'avg s valence electrons', 'avg p valence electrons',
       'avg d valence electrons', 'avg f valence electrons',
       'frac s valence electrons', 'frac p valence electrons',
       'frac d valence electrons', 'frac f valence electrons',
       'Average Mulliken electronegativity', 'Valence electrons',
       'Valence Electrons Density avg', 'Oxygen_concentration avg',
       'Packing fraction avg','Log_rate']

In [84]:
categorical_cols = [col for col in columns_EDA
                    if df[col].dtype in ['object', 'category','bool']]

numerical_cols = [col for col in columns_EDA
                  if df[col].dtype in ['int64', 'float64']]
print(categorical_cols)
print(numerical_cols)

['Perovskite', 'Symmetry group', 'Prep Method', 'Nitrogen', 'Promotion method', 'Promoter', 'Light type', 'Cell material']
['Bandgap, eV', 'd,A', 'Number of octahedrons on a layer', 'CalcT(K)', 'Calc time (h)', 'Promoter, w%', 'Surface area, m2/g', 'CatW, g/L', 'Alcohol, %', 'Power, W', 'Wave length min, nm', 'Temperature, K', 'Rate, umol/(g*h)', 'avg s valence electrons', 'avg p valence electrons', 'avg d valence electrons', 'avg f valence electrons', 'frac s valence electrons', 'frac p valence electrons', 'frac d valence electrons', 'frac f valence electrons', 'Average Mulliken electronegativity', 'Valence electrons', 'Valence Electrons Density avg', 'Oxygen_concentration avg', 'Packing fraction avg', 'Log_rate']


In [85]:
import plotly.express as px
from scipy.stats import gaussian_kde
import plotly.graph_objects as go

def plot_smooth_distribution(df, column):
    data = df[column].dropna().values

    kde = gaussian_kde(data)
    x_vals = np.linspace(data.min(), data.max(), 500)
    y_vals = kde(x_vals)

    # Plot
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=x_vals,
        y=y_vals,
        mode='lines',
        line=dict(width=3),
        name=f'{column} KDE'
    ))

    fig.update_layout(
        title=f"Distribution of '{column}'",
        xaxis_title=column,
        yaxis_title="Density",
        #template="plotly_white"
    )

    fig.show()

In [86]:
plot_smooth_distribution(df, 'Bandgap, eV')

In [87]:
for col in numerical_cols:
    fig = px.histogram(df, x=col, nbins=20, title=f'Histogram of {col}',width=800, height=500)
    fig.show()

In [88]:
def draw_pie_chart(df, column):
  fig = px.pie(df, names=column, title=f'{column} Distribution',hole=0.3)
  fig.update_traces(textinfo='percent+label')
  fig.show()

In [ ]:
for col in categorical_cols:
     draw_pie_chart(df,col)

In [89]:
from scipy.stats import pearsonr

def draw_scatter_plot(df,x_column, y_column):
  from sklearn.metrics import r2_score

  x = df[x_column]
  y = df[y_column]

  r, p_value = pearsonr(x, y)

  fig = go.Figure()

  fig.add_trace(go.Scatter(
      x=x,
      y=y,
      mode='markers',
      name='Data',
      marker=dict(size=8, color='blue', opacity=0.7)
  ))

  fig.add_annotation(
    x=np.mean(x),
    y=np.max(y),
    text=f"Pearson r = {r:.3f}",
    showarrow=False,
    font=dict(size=14, color="red")
  )

  fig.update_layout(
      xaxis_title=f'{x_column}',
      yaxis_title=f'{y_column}',
      template="plotly_white",
      width=600,
      height=600,
  )

  fig.show()

In [90]:
for col in numerical_cols:
  draw_scatter_plot(df,col,'Rate, umol/(g*h)')

In [ ]:
for col in numerical_cols:
  draw_scatter_plot(df,col,target_col)

In [91]:
df_selected = df[numerical_cols]

def draw_corr_heatmap(_df):
  # Compute correlation matrix
  corr_matrix = _df.corr()

  # Plot heatmap
  fig = px.imshow(
      corr_matrix,
      text_auto=True,        # show correlation values
      aspect="auto",
      color_continuous_scale='RdBu_r',  # diverging color map
      title="Correlation Heatmap",
      width=1000,
      height=1000,
  )
  fig.show()

draw_corr_heatmap(df_selected)

In [92]:
df.to_excel('checkpoint_descriptors.xlsx')

#Preparing data for ML

##Categorization


In [93]:
#raise SystemExit("Stopping execution")

In [94]:
categorical_columns = df.select_dtypes(include='object').columns.tolist()
categorical_columns.remove('Perovskite')
categorical_columns.remove('Interlayer space composition')
categorical_columns.remove('Materials Project ID')
categorical_columns.remove('COD_ID')
categorical_columns.remove('Springer_ID')
categorical_columns.remove('Symmetry group')
categorical_columns.remove('Sac Agent 1')
categorical_columns.remove('Sac agent 2')
categorical_columns.remove('Hill formula')
categorical_columns

['Prep Method', 'Promotion method', 'Promoter', 'Light type', 'Cell material']

In [95]:
df2 = df['Light type'].value_counts()
print(df2.index)

Index(['Hg lamp', 'Xe lamp'], dtype='object', name='Light type')


In [96]:
df[categorical_columns].isna().sum()

,0
Prep Method,0
Promotion method,0
Promoter,0
Light type,0
Cell material,0


In [97]:
df = pd.get_dummies(df, columns=categorical_columns, drop_first=True)
df.columns


Index(['Perovskite', 'Interlayer space composition', 'Bandgap, eV',
       'Materials Project ID', 'COD_ID', 'Springer_ID', 'Z', 'a, A', 'b, A',
       'c, A', 'Symmetry group', 'd,A', 'Number of octahedrons on a layer',
       'CalcT(K)', 'Calc time (h)', 'Nitrogen', 'Promoter, w%',
       'Surface area, m2/g', 'CatW, g/L', 'Alcohol, %', 'Sac Agent 1',
       'Sac agent 2', 'Power, W', 'Wave length min, nm', 'Temperature, K',
       'Rate, umol/(g*h)', 'Hill formula', 'avg s valence electrons',
       'avg p valence electrons', 'avg d valence electrons',
       'avg f valence electrons', 'frac s valence electrons',
       'frac p valence electrons', 'frac d valence electrons',
       'frac f valence electrons', 'Average Mulliken electronegativity',
       'Valence electrons', 'Volume_MP', 'Z_MP',
       'Valence Electrons Density_MP', 'a_MP', 'b_MP', 'c_MP', 'Volume_COD',
       'Z_COD', 'Valence Electrons Density_COD', 'a_COD', 'b_COD', 'c_COD',
       'Volume_Springer', 'Z_Springer'

In [98]:
columns_for_ML = df.columns
columns_to_exlude = ['Perovskite', 'Interlayer space composition','Materials Project ID', 'COD_ID', 'Springer_ID', 'Z', 'a, A', 'b, A',
       'c, A', 'Symmetry group', 'Number of octahedrons on a layer','Sac Agent 1',
       'Sac agent 2','Rate, umol/(g*h)','Hill formula','Volume_MP', 'Z_MP',
       'Valence Electrons Density_MP', 'a_MP', 'b_MP', 'c_MP', 'Volume_COD',
       'Z_COD', 'Valence Electrons Density_COD', 'a_COD', 'b_COD', 'c_COD',
       'Volume_Springer', 'Z_Springer', 'Valence Electrons Density_Springer',
       'a_Springer', 'b_Springer', 'c_Springer', 'Volume_manual',
       'Valence Electrons Density_manual', 'Oxygen_count',
       'Oxygen_concentration_manual', 'Oxygen_concentration_MP',
       'Oxygen_concentration_COD', 'Oxygen_concentration_Springer',
       'MP_packing_fraction', 'COD_packing_fraction',
       'Springer_packing_fraction', 'Manual_packing_fraction']

columns_for_ML = [x for x in columns_for_ML if x not in columns_to_exlude]
print(columns_for_ML)


['Bandgap, eV', 'd,A', 'CalcT(K)', 'Calc time (h)', 'Nitrogen', 'Promoter, w%', 'Surface area, m2/g', 'CatW, g/L', 'Alcohol, %', 'Power, W', 'Wave length min, nm', 'Temperature, K', 'avg s valence electrons', 'avg p valence electrons', 'avg d valence electrons', 'avg f valence electrons', 'frac s valence electrons', 'frac p valence electrons', 'frac d valence electrons', 'frac f valence electrons', 'Average Mulliken electronegativity', 'Valence electrons', 'Valence Electrons Density avg', 'Oxygen_concentration avg', 'Packing fraction avg', 'Log_rate', 'Prep Method_PC', 'Prep Method_PTP', 'Prep Method_SG', 'Prep Method_SSR', 'Prep Method_co-pre', 'Prep Method_perchased', 'Prep Method_purchased', 'Promotion method_PD', 'Promotion method_impregnation', 'Promoter_CuO', 'Promoter_Fe2O3', 'Promoter_Ni', 'Promoter_NiO', 'Promoter_No promoter', 'Promoter_Pt', 'Promoter_Rh', 'Promoter_RuO2', 'Promoter_ZnO', 'Light type_Xe lamp', 'Cell material_quartz']


In [99]:
df = df[columns_for_ML]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 478 entries, 19 to 1088
Data columns (total 46 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Bandgap, eV                         478 non-null    float64
 1   d,A                                 390 non-null    float64
 2   CalcT(K)                            478 non-null    float64
 3   Calc time (h)                       478 non-null    float64
 4   Nitrogen                            478 non-null    bool   
 5   Promoter, w%                        478 non-null    float64
 6   Surface area, m2/g                  478 non-null    float64
 7   CatW, g/L                           478 non-null    float64
 8   Alcohol, %                          478 non-null    float64
 9   Power, W                            478 non-null    float64
 10  Wave length min, nm                 478 non-null    float64
 11  Temperature, K                      478 non-null

In [100]:
df["Valence Electrons Density avg"] = df["Valence Electrons Density avg"].fillna(df["Valence Electrons Density avg"].mean())
df["Oxygen_concentration avg"] = df["Oxygen_concentration avg"].fillna(df["Oxygen_concentration avg"].mean())
df["Packing fraction avg"] = df["Packing fraction avg"].fillna(df["Packing fraction avg"].mean())

In [101]:
df["d,A"] = df["d,A"].fillna(df["d,A"].mean())
print(df["d,A"].mean())

16.007748230769227


In [102]:
df.isna().sum().sum()

np.int64(0)

In [103]:
draw_corr_heatmap(df)

In [104]:
features_to_remove=['Oxygen_concentration avg','Valence electrons']
df = df.drop(columns=features_to_remove)

##Normalization

In [105]:
from sklearn.preprocessing import MinMaxScaler

features = list(df.columns)
features.remove('Log_rate')

scaler = MinMaxScaler()

df_normalized = df.copy()
df_normalized[features] = scaler.fit_transform(df_normalized[features])
df_normalized.head()

,"Bandgap, eV","d,A",CalcT(K),Calc time (h),Nitrogen,"Promoter, w%","Surface area, m2/g","CatW, g/L","Alcohol, %","Power, W",...,Promoter_Fe2O3,Promoter_Ni,Promoter_NiO,Promoter_No promoter,Promoter_Pt,Promoter_Rh,Promoter_RuO2,Promoter_ZnO,Light type_Xe lamp,Cell material_quartz
19,0.2500,0.000000,0.678085,0.470028,0.0,0.05,0.096577,0.927553,0.158889,0.866667,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
69,0.8125,0.319540,0.750000,0.245283,0.0,0.00,0.096577,0.318339,0.000000,0.733333,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
70,0.5625,0.319540,0.750000,0.245283,0.0,0.00,0.096577,0.318339,0.000000,0.733333,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
71,0.7500,0.318966,0.750000,0.245283,0.0,0.00,0.096577,0.318339,0.000000,0.733333,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
72,0.7500,0.319253,0.750000,0.245283,0.0,0.00,0.096577,0.318339,0.000000,0.733333,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [106]:
draw_corr_heatmap(df_normalized)

In [107]:
X = df_normalized.drop(columns=[target_col])
y = df_normalized[target_col]

corr_matrix = X.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
threshold = 0.9
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
print("Columns to drop:", to_drop)
df_reduced = df.drop(columns=to_drop)

Columns to drop: ['Wave length min, nm', 'Temperature, K', 'frac d valence electrons', 'frac f valence electrons', 'Promoter_Pt', 'Promoter_RuO2']


In [108]:
df_normalized.to_excel("checkpoint_ML_input.xlsx")

#ML


In [109]:
#raise SystemExit("Stopping execution")


print('Std:',y.std())

print(X.shape[0])

Std: 2.9240017711889217
478


In [110]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import KFold

In [111]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=True
)


In [112]:
def draw_actual_predicted(_y_test, _y_pred):
  fig = px.scatter(
    x=_y_test,
    y=_y_pred,
    labels={'x':'Actual', 'y':'Predicted'},
    title='Actual vs Predicted',
    width = 700,
    height = 700,
  )
  fig.add_shape(
      type="line",
      x0=_y_test.min(), y0=_y_pred.min(),
      x1=_y_test.max(), y1=_y_pred.max(),
      line=dict(color="red", dash="dash")
  )
  rmse = np.sqrt(mean_squared_error(_y_test, _y_pred))
  r2 = r2_score(_y_test, _y_pred)

  fig.add_annotation(
    x=0.05, y=0.95, xref="paper", yref="paper",
    text=f"R² = {r2:.4f}<br>RMSE = {rmse:.4f}",
    showarrow=False,
    font=dict(size=14)
  )
  fig.show()

##Linear regression

In [113]:
linear_model = LinearRegression()

if(cross_validation):
  cv = KFold(n_splits=5, shuffle=True)
  scores = cross_val_score(linear_model, X, y, cv=cv, scoring='neg_mean_squared_error')
  rmse_scores = np.sqrt(-scores)
  print("Scores: ", scores)
  print("Mean RMSE:", rmse_scores.mean())

  y_pred = cross_val_predict(linear_model, X, y, cv=5)
  draw_actual_predicted(y, y_pred)
else:
  print('No cross validation')
  linear_model.fit(X_train, y_train)
  y_pred = linear_model.predict(X_test)
  draw_actual_predicted(y_test, y_pred)

Scores:  [-2.96773052 -3.15717515 -3.15518809 -3.52071722 -2.46824645]
Mean RMSE: 1.7446524224672948


In [114]:
def estimate_performance(_y_test, _y_pred):
  raisemse = np.sqrt(mean_squared_error(_y_test, _y_pred))
  r2 = r2_score(_y_test, _y_pred)

  print("RMSE:", mse)
  print("R² Score:", r2)

##Decision tree regression

In [115]:
from sklearn.tree import DecisionTreeRegressor

In [116]:
tree_model = DecisionTreeRegressor(random_state=42)

if(cross_validation):
  scores = cross_val_score(tree_model, X, y, cv=5, scoring='neg_mean_squared_error')
  rmse_scores = np.sqrt(-scores)
  print("Scores: ", scores)
  print("Mean RMSE:", rmse_scores.mean())

  y_pred = cross_val_predict(tree_model, X, y, cv=5)
  draw_actual_predicted(y, y_pred)
else:
  print('No cross validation')
  tree_model.fit(X_train, y_train)
  y_pred = tree_model.predict(X_test)
  draw_actual_predicted(y_test, y_pred)

Scores:  [ -7.4521137   -6.3141831   -8.16753658  -4.21786691 -10.72638904]
Mean RMSE: 2.6858822606256862


In [117]:
import matplotlib.pyplot as plt

if not(cross_validation):
  feat_imp = pd.DataFrame({
      'Feature': X.columns,
      'Importance': tree_model.feature_importances_
  }).sort_values(by='Importance', ascending=False)

  plt.figure(figsize=(8,6))
  sns.barplot(x='Importance', y='Feature', data=feat_imp)
  plt.title('Feature Importance')
  plt.show()

## Random forest

In [118]:
from sklearn.ensemble import RandomForestRegressor

In [119]:
rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=3,
    min_samples_split = 5,
    min_samples_leaf = 3,
    max_features=2,
    random_state=42
)

if(cross_validation):
  scores = cross_val_score(rf_model , X, y, cv=5, scoring='neg_mean_squared_error')
  rmse_scores = np.sqrt(-scores)
  print("Scores: ", scores)
  print("Mean RMSE:", rmse_scores.mean())

  y_pred = cross_val_predict(rf_model , X, y, cv=5)
  draw_actual_predicted(y, y_pred)
else:
  print('No cross validation')
  rf_model.fit(X_train, y_train)
  y_pred = rf_model.predict(X_test)
  draw_actual_predicted(y_test, y_pred)

Scores:  [-6.00985711 -4.50464985 -3.83267714 -5.29953041 -5.88145264]
Mean RMSE: 2.2517762005137807


In [120]:
if not(cross_validation):
  feat_imp = pd.DataFrame({
      'Feature': X.columns,
      'Importance': rf_model.feature_importances_
  }).sort_values(by='Importance', ascending=False)

  plt.figure(figsize=(8,6))
  sns.barplot(x='Importance', y='Feature', data=feat_imp)
  plt.title('Feature Importance')
  plt.show()

In [121]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [3, 4, 5],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 3, 5],
    'max_features': [1, 2, 3]
}

grid_search = GridSearchCV(RandomForestRegressor(random_state=42),
                           param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X,y)
print('Best params',grid_search.best_params_)
print('Best score',grid_search.best_score_)

Best params {'max_depth': 5, 'max_features': 2, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Best score -4.387236683768616


In [122]:
best_model = grid_search.best_estimator_

if(cross_validation):
  scores = cross_val_score(best_model , X, y, cv=5, scoring='neg_mean_squared_error')
  rmse_scores = np.sqrt(-scores)
  print("Scores: ", scores)
  print("Mean RMSE:", rmse_scores.mean())

  y_pred = cross_val_predict(best_model , X, y, cv=5)
  draw_actual_predicted(y, y_pred)
else:
  y_pred = best_model.predict(X_test)
  draw_actual_predicted(y_test, y_pred)

Scores:  [-4.56446009 -4.42217924 -4.12280388 -3.77284208 -5.05389812]
Mean RMSE: 2.092058918065272


##Gradient boosting

In [123]:
from sklearn.ensemble import GradientBoostingRegressor

In [124]:
gb_model = GradientBoostingRegressor(
    n_estimators=100,      # number of boosting stages
    learning_rate=0.1,     # shrinkage factor
    max_depth=3,           # depth of each tree
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42
)

if(cross_validation):
  scores = cross_val_score(gb_model , X, y, cv=5, scoring='neg_mean_squared_error')
  rmse_scores = np.sqrt(-scores)
  print("Scores: ", scores)
  print("Mean RMSE:", rmse_scores.mean())

  y_pred = cross_val_predict(gb_model , X, y, cv=5)
  draw_actual_predicted(y, y_pred)
else:
  gb_model.fit(X_train, y_train)
  y_pred = gb_model.predict(X_test)
  draw_actual_predicted(y_test, y_pred)

Scores:  [-4.11735078 -4.29233873 -3.55203917 -2.64324009 -6.300566  ]
Mean RMSE: 2.024300867240233


In [125]:
if not(cross_validation):
  feat_imp = pd.DataFrame({
      'Feature': X.columns,
      'Importance': gb_model.feature_importances_
  }).sort_values(by='Importance', ascending=False)

  plt.figure(figsize=(8,6))
  sns.barplot(x='Importance', y='Feature', data=feat_imp)
  plt.title('Feature Importance')
  plt.show()

In [126]:
param_grid = {
    'n_estimators': [100, 150],        # number of boosting stages
    'learning_rate': [0.01, 0.1],   # shrinkage factor
    'max_depth': [2, 3, 4],                # depth of each tree
    'min_samples_split': [2, 4, 6],
    'min_samples_leaf': [1, 2, 3],
    'subsample': [0.8, 1.0]                # fraction of samples for each tree
}

# Grid search with 5-fold cross-validation
grid_search = GridSearchCV(
    estimator=gb_model,
    param_grid=param_grid,
    cv=5,
    scoring='neg_mean_squared_error',          # or 'neg_mean_squared_error'
    n_jobs=-1,
    verbose=2
)

# Fit on training data
grid_search.fit(X, y)

# Best hyperparameters
print("Best parameters:", grid_search.best_params_)
print("Best CV RMSE:", grid_search.best_score_)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Best parameters: {'learning_rate': 0.1, 'max_depth': 2, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 150, 'subsample': 0.8}
Best CV RMSE: -3.9188706417402495


In [127]:
best_model = grid_search.best_estimator_

if(cross_validation):
  scores = cross_val_score(best_model , X, y, cv=5, scoring='neg_mean_squared_error')
  rmse_scores = np.sqrt(-scores)
  print("Scores: ", scores)
  print("Mean RMSE:", rmse_scores.mean())

  y_pred = cross_val_predict(best_model , X, y, cv=5)
  draw_actual_predicted(y, y_pred)
else:
  y_pred = best_model.predict(X_test)
  draw_actual_predicted(y_test, y_pred)

Scores:  [-4.44102014 -3.93887269 -3.28548501 -2.19866496 -5.73031041]
Mean RMSE: 1.9562438196129261


##KNN

In [128]:
from sklearn.neighbors import KNeighborsRegressor

In [129]:
knn_model = KNeighborsRegressor(n_neighbors=5)

if(cross_validation):
  scores = cross_val_score(knn_model , X, y, cv=5, scoring='neg_mean_squared_error')
  rmse_scores = np.sqrt(-scores)
  print("Scores: ", scores)
  print("Mean RMSE:", rmse_scores.mean())

  y_pred = cross_val_predict(knn_model , X, y, cv=5)
  draw_actual_predicted(y, y_pred)
else:
  knn_model.fit(X_train, y_train)
  y_pred = knn_model.predict(X_test)
  draw_actual_predicted(y_test, y_pred)

Scores:  [-6.34609835 -8.42329252 -7.35248155 -6.04802902 -4.59406416]
Mean RMSE: 2.547126825465468


In [130]:
param_grid = {
    'n_neighbors': list(range(1, 21)),
    'weights': ['uniform', 'distance'],
    'p': [1, 2]
}

grid_search = GridSearchCV(
    KNeighborsRegressor(),
    param_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=2
)

grid_search.fit(X,y)

print("Best parameters:", grid_search.best_params_)
print("Best CV RMSE:", grid_search.best_score_)

Fitting 5 folds for each of 80 candidates, totalling 400 fits
Best parameters: {'n_neighbors': 20, 'p': 1, 'weights': 'distance'}
Best CV RMSE: -4.945732731680516


In [131]:
best_model = grid_search.best_estimator_

if(cross_validation):
  scores = cross_val_score(best_model , X, y, cv=5, scoring='neg_mean_squared_error')
  rmse_scores = np.sqrt(-scores)
  print("Scores: ", scores)
  print("Mean RMSE:", rmse_scores.mean())

  y_pred = cross_val_predict(best_model , X, y, cv=5)
  draw_actual_predicted(y, y_pred)
else:
  y_pred = best_model.predict(X_test)
  draw_actual_predicted(y_test, y_pred)

Scores:  [-5.3956805  -5.59477323 -5.37569032 -4.25452797 -4.10799164]
Mean RMSE: 2.219242006181706


##Catboost

In [132]:
!pip install catboost
from catboost import CatBoostRegressor

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 5.8 MB/s eta 0:00:00


In [133]:
cat_boost_model = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    verbose=100
)

if(cross_validation):
  scores = cross_val_score(cat_boost_model , X, y, cv=5, scoring='neg_mean_squared_error')
  rmse_scores = np.sqrt(-scores)
  print("Scores: ", scores)
  print("Mean RMSE:", rmse_scores.mean())

  y_pred = cross_val_predict(cat_boost_model , X, y, cv=5)
  draw_actual_predicted(y, y_pred)
else:
  cat_boost_model.fit(X_train, y_train)
  y_pred = cat_boost_model.predict(X_test)
  draw_actual_predicted(y_test, y_pred)

0:	learn: 2.6813426	total: 62.2ms	remaining: 1m 2s
100:	learn: 0.7071746	total: 513ms	remaining: 4.56s
200:	learn: 0.4747313	total: 964ms	remaining: 3.83s
300:	learn: 0.3342692	total: 1.14s	remaining: 2.65s
400:	learn: 0.2589748	total: 1.56s	remaining: 2.33s
500:	learn: 0.2089274	total: 2.25s	remaining: 2.25s
600:	learn: 0.1805797	total: 3.03s	remaining: 2.01s
700:	learn: 0.1628715	total: 3.74s	remaining: 1.6s
800:	learn: 0.1472095	total: 4.85s	remaining: 1.21s
900:	learn: 0.1360365	total: 5.65s	remaining: 621ms
999:	learn: 0.1280940	total: 6.41s	remaining: 0us
0:	learn: 2.8439902	total: 18.3ms	remaining: 18.3s
100:	learn: 0.7103216	total: 736ms	remaining: 6.55s
200:	learn: 0.4689263	total: 1.3s	remaining: 5.17s
300:	learn: 0.3414936	total: 2s	remaining: 4.64s
400:	learn: 0.2614025	total: 2.66s	remaining: 3.98s
500:	learn: 0.2071799	total: 3.33s	remaining: 3.32s
600:	learn: 0.1719868	total: 3.75s	remaining: 2.49s
700:	learn: 0.1476047	total: 4.04s	remaining: 1.72s
800:	learn: 0.1283593

In [134]:
param_grid = {
    'depth': [4,6,8],
    'learning_rate':[0.01,0.05,0.1 ]
}

grid_search = GridSearchCV(cat_boost_model, param_grid, cv=5,scoring='neg_mean_squared_error',)
grid_search.fit(X,y)

print("Best parameters:", grid_search.best_params_)
print("Best CV RMSE:", grid_search.best_score_)

0:	learn: 2.8315699	total: 970us	remaining: 969ms
100:	learn: 1.8740496	total: 213ms	remaining: 1.89s
200:	learn: 1.5097569	total: 700ms	remaining: 2.78s
300:	learn: 1.3425336	total: 1.25s	remaining: 2.91s
400:	learn: 1.2383510	total: 1.67s	remaining: 2.49s
500:	learn: 1.1619378	total: 2.02s	remaining: 2.01s
600:	learn: 1.0994182	total: 2.47s	remaining: 1.64s
700:	learn: 1.0472485	total: 2.99s	remaining: 1.28s
800:	learn: 1.0030454	total: 3.22s	remaining: 799ms
900:	learn: 0.9675847	total: 3.48s	remaining: 383ms
999:	learn: 0.9319592	total: 3.89s	remaining: 0us
0:	learn: 2.9866767	total: 6.5ms	remaining: 6.49s
100:	learn: 1.9362244	total: 365ms	remaining: 3.25s
200:	learn: 1.5288480	total: 748ms	remaining: 2.97s
300:	learn: 1.3433744	total: 1.04s	remaining: 2.41s
400:	learn: 1.2403991	total: 1.38s	remaining: 2.06s
500:	learn: 1.1621892	total: 1.75s	remaining: 1.74s
600:	learn: 1.0928866	total: 2.08s	remaining: 1.38s
700:	learn: 1.0383189	total: 2.25s	remaining: 960ms
800:	learn: 0.9862

In [135]:
best_model = grid_search.best_estimator_

if(cross_validation):
  print("Cross validation")
  scores = cross_val_score(best_model , X, y, cv=5, scoring='neg_mean_squared_error')
  rmse_scores = np.sqrt(-scores)

  y_pred = cross_val_predict(best_model , X, y, cv=5)
  draw_actual_predicted(y, y_pred)
  print("Scores: ", scores)
  print("Mean RMSE:", rmse_scores.mean())
else:
  y_pred = best_model.predict(X_test)
  draw_actual_predicted(y_test, y_pred)

Cross validation
0:	learn: 2.7642617	total: 2.3ms	remaining: 2.3s
100:	learn: 0.9653620	total: 194ms	remaining: 1.73s
200:	learn: 0.7259804	total: 348ms	remaining: 1.38s
300:	learn: 0.5642852	total: 515ms	remaining: 1.2s
400:	learn: 0.4646616	total: 680ms	remaining: 1.01s
500:	learn: 0.3959892	total: 845ms	remaining: 841ms
600:	learn: 0.3367751	total: 1s	remaining: 665ms
700:	learn: 0.2910211	total: 1.19s	remaining: 507ms
800:	learn: 0.2572797	total: 1.34s	remaining: 334ms
900:	learn: 0.2314683	total: 1.51s	remaining: 166ms
999:	learn: 0.2103545	total: 1.67s	remaining: 0us
0:	learn: 2.9226897	total: 2.33ms	remaining: 2.33s
100:	learn: 0.9555204	total: 183ms	remaining: 1.63s
200:	learn: 0.7003490	total: 384ms	remaining: 1.53s
300:	learn: 0.5613368	total: 574ms	remaining: 1.33s
400:	learn: 0.4502216	total: 759ms	remaining: 1.13s
500:	learn: 0.3817641	total: 944ms	remaining: 940ms
600:	learn: 0.3330775	total: 1.13s	remaining: 753ms
700:	learn: 0.2934538	total: 1.32s	remaining: 561ms
800:	

Scores:  [-3.3188063  -3.8055891  -3.92134376 -2.99836299 -5.37929431]
Mean RMSE: 1.9607396203586673


In [136]:
y_pred = cross_val_predict(best_model , X, y, cv=5)

0:	learn: 2.7642617	total: 1.99ms	remaining: 1.99s
100:	learn: 0.9653620	total: 190ms	remaining: 1.69s
200:	learn: 0.7259804	total: 356ms	remaining: 1.41s
300:	learn: 0.5642852	total: 525ms	remaining: 1.22s
400:	learn: 0.4646616	total: 734ms	remaining: 1.1s
500:	learn: 0.3959892	total: 1.1s	remaining: 1.1s
600:	learn: 0.3367751	total: 1.56s	remaining: 1.04s
700:	learn: 0.2910211	total: 2.06s	remaining: 878ms
800:	learn: 0.2572797	total: 2.43s	remaining: 603ms
900:	learn: 0.2314683	total: 2.93s	remaining: 322ms
999:	learn: 0.2103545	total: 3.38s	remaining: 0us
0:	learn: 2.9226897	total: 4.59ms	remaining: 4.59s
100:	learn: 0.9555204	total: 505ms	remaining: 4.5s
200:	learn: 0.7003490	total: 988ms	remaining: 3.92s
300:	learn: 0.5613368	total: 1.35s	remaining: 3.13s
400:	learn: 0.4502216	total: 1.53s	remaining: 2.28s
500:	learn: 0.3817641	total: 1.72s	remaining: 1.71s
600:	learn: 0.3330775	total: 1.93s	remaining: 1.28s
700:	learn: 0.2934538	total: 2.12s	remaining: 904ms
800:	learn: 0.258420

In [137]:
draw_actual_predicted(y, y_pred)